<a href="https://colab.research.google.com/github/savithashivakumaar-debug/savitha/blob/main/Copy_of_Nutrition1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Nutrition Paradox: A Global View on Obesity and Malnutrition

### Step 1:Dataset Overview & Collection

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

For Obesity:

In [ ]:
url = "https://ghoapi.azureedge.net/api/NCD_BMI_30C"  #adults

import requests

response = requests.get(url)

data = response.json()

data

In [ ]:
data.keys()

In [ ]:
import pandas as pd
df1 = pd.DataFrame(data['value'])
df1.head()

In [ ]:
df1.shape

# Add a new column age_group to distinguish adults and children.

In [ ]:
df1["age_group"] = "adults"

In [ ]:
df1.head()

In [ ]:
df1.columns

### children

In [ ]:
url = "https://ghoapi.azureedge.net/api/NCD_BMI_PLUS2C"    #children

import requests

response = requests.get(url)

data = response.json()

data

In [ ]:
data.keys()

In [ ]:
import pandas as pd
df2 = pd.DataFrame(data['value'])
df2.head()

In [ ]:
df2.shape

In [ ]:
df2["age_group"] = "children"

In [ ]:
df2.head()

In [ ]:
df2.columns

### Combine the two obesity datasets into one dataframe called df_obesity.

In [ ]:
import pandas as pd

# Step 1: Use the existing dataframes df1 and df2
# df1 = pd.read_csv('obesity_data1.csv')
# df2 = pd.read_csv('obesity_data2.csv')

# Step 2: Combine (merge or concatenate) them into one dataframe
df_obesity = pd.concat([df1, df2], ignore_index=True)

# Step 3: Display the combined dataframe
print(df_obesity.head())

### Filter each dataset to include only records from the years 2012 to 2022

In [ ]:
# Filter df1 for years 2012–2022
df1_filtered = df1[(df1['TimeDim'] >= 2012) & (df1['TimeDim'] <= 2022)]

# Filter df2 for years 2012–2022
df2_filtered = df2[(df2['TimeDim'] >= 2012) & (df2['TimeDim'] <= 2022)]


In [ ]:
import pandas as pd

# Combine the filtered datasets
df_obesity = pd.concat([df1_filtered, df2_filtered], ignore_index=True)

# Optional: check
print(df_obesity['TimeDim'].min(), df_obesity['TimeDim'].max())  # Should show 2012 and 2022
print(df_obesity.shape)  # Number of rows and columns


# Step 2: 🧹 Data Cleaning & Feature Engineering

In [ ]:
import pandas as pd

columns_needed = [
    "ParentLocation",
    "Dim1",
    "TimeDim",
    "Low",
    "High",
    "NumericValue",
    "SpatialDim",
    "age_group"
]

df2_filtered = df2[columns_needed]

print(df2_filtered.head())


### Columns to Retain:

In [ ]:
import pandas as pd

columns_needed = [
    "ParentLocation",
    "Dim1",
    "TimeDim",
    "Low",
    "High",
    "NumericValue",
    "SpatialDim",
    "age_group"
]

# Filter columns
df1_filtered = df1[columns_needed]
df2_filtered = df2[columns_needed]

# Keep only rows where TimeDim is between 2012 and 2022
df1_filtered = df1_filtered[(df1_filtered["TimeDim"] >= 2012) & (df1_filtered["TimeDim"] <= 2022)]
df2_filtered = df2_filtered[(df2_filtered["TimeDim"] >= 2012) & (df2_filtered["TimeDim"] <= 2022)]

# Combine into one dataframe
df_obesity = pd.concat([df1_filtered, df2_filtered], ignore_index=True)

print(df_obesity.head())
print("\nShape of combined dataset:", df_obesity.shape)



### Rename Columns for Consistency:

In [ ]:
# Rename columns for consistency
df_obesity = df_obesity.rename(columns={
    "TimeDim": "Year",
    "Dim1": "Gender",
    "NumericValue": "Mean_Estimate",
    "Low": "LowerBound",
    "High": "UpperBound",
    "ParentLocation": "Region",
    "SpatialDim": "Country"
})

# Standardize Gender values
gender_map = {
    "Both sexes": "Both",
    "BTSX": "Both",
    "Male": "Male",
    "MLE": "Male",
    "Female": "Female",
    "FMLE": "Female"
}

df_obesity["Gender"] = df_obesity["Gender"].replace(gender_map)

print(df_obesity.head())
print("\nUnique Gender values:", df_obesity["Gender"].unique())


### Convert Country Codes to Full Names using pycountry:

# Install ‘pycountry’ package

In [ ]:
!pip install pycountry

# Import the package in your Python script or notebook

In [ ]:
import pycountry

special_cases = {
    'GLOBAL': 'Global',
    'WB_LMI': 'Low & Middle Income',
    'WB_HI': 'High Income',
    'WB_LI': 'Low Income',
    'EMR': 'Eastern Mediterranean Region',
    'EUR': 'Europe',
    'AFR': 'Africa',
    'SEAR': 'South-East Asia Region',
    'WPR': 'Western Pacific Region',
    'AMR': 'Americas Region',
    'WB_UMI': 'Upper Middle Income'
}

def convert_country(code):
    # Handle special cases first
    if code in special_cases:
        return special_cases[code]

    # Try to look up in pycountry using Alpha-3 code
    try:
        return pycountry.countries.get(alpha_3=code).name
    except AttributeError:
        # If Alpha-3 fails, try Alpha-2
        try:
            return pycountry.countries.get(alpha_2=code).name
        except AttributeError:
            return code  # If no match found, keep original

## Define a function to convert the 3-letter codes to full names

In [ ]:
import pycountry
print(convert_country('AFR'))

## Apply the function to the Country colum

In [ ]:
import pandas as pd
import pycountry

# Example data
df = pd.DataFrame({
    'Country': ['IND', 'USA', 'AFR', 'FRA', 'CHN']
})

# Function to convert 3-letter codes to full names (handles regions too)
def convert_country(code):
    regions = {
        'AFR': 'Africa',
        'EUR': 'Europe',
        'ASI': 'Asia',
        'AME': 'America',
        'OCE': 'Oceania'
    }
    code = str(code).upper()

    if code in regions:
        return regions[code]

    country = pycountry.countries.get(alpha_3=code)
    return country.name if country else "Invalid country code"

# ✅ Apply the function to the Country column
df['Country_Full'] = df['Country'].apply(convert_country)

print(df)


Handle any remaining unmatched codes using the following dictionary for special values like 'GLOBAL', 'AFR', 'SEAR', etc.

In [ ]:
import pandas as pd
import pycountry

# Example DataFrame
df = pd.DataFrame({
    'Country': ['IND', 'USA', 'AFR', 'GLOBAL', 'SEAR', 'EUR', 'XYZ']
})

# Dictionary for special or regional codes
special_regions = {
    'GLOBAL': 'Global',
    'AFR': 'Africa',
    'AMR': 'Americas',
    'EMR': 'Eastern Mediterranean Region',
    'EUR': 'Europe',
    'SEAR': 'South-East Asia Region',
    'WPR': 'Western Pacific Region'
}

# Function to convert country/region codes to full names
def convert_country(code):
    if not code:
        return "Unknown"

    code = str(code).upper()

    # 1️⃣ Check special codes
    if code in special_regions:
        return special_regions[code]

    # 2️⃣ Try pycountry for standard ISO 3-letter country codes
    country = pycountry.countries.get(alpha_3=code)
    if country:
        return country.name

    # 3️⃣ Fallback for unmatched codes
    return "Unmatched"

# ✅ Apply the function
df['Country_Full'] = df['Country'].apply(convert_country)

print(df)

### New Columns to Create:

# 1. age_group: Manually assign this column based on the dataset source.
*Use "Adult" for datasets NCD_BMI_30C and NCD_BMI_18C.

In [ ]:
import pandas as pd

# Example DataFrames (already loaded separately)
df_30C = pd.DataFrame({'Dataset': ['NCD_BMI_30C'] * 3, 'Value': [25, 27, 29]})
df_18C = pd.DataFrame({'Dataset': ['NCD_BMI_18C'] * 3, 'Value': [20, 22, 23]})

# ✅ Assign manually based on dataset
df_30C['age_group'] = 'Adult'
df_18C['age_group'] = 'Adult'

# Combine if needed
df_combined = pd.concat([df_30C, df_18C], ignore_index=True)
print(df_combined)


Use "Child/Adolescent" for datasets NCD_BMI_PLUS2C and NCD_BMI_MINUS2C.

In [ ]:
import pandas as pd

# Example combined dataset
df = pd.DataFrame({
    'Dataset': ['NCD_BMI_30C', 'NCD_BMI_18C', 'NCD_BMI_PLUS2C', 'NCD_BMI_MINUS2C'],
    'Value': [25.3, 22.7, 19.4, 17.8]
})

# ✅ Assign age group based on dataset name
df['age_group'] = df['Dataset'].map({
    'NCD_BMI_30C': 'Adult',
    'NCD_BMI_18C': 'Adult',
    'NCD_BMI_PLUS2C': 'Child/Adolescent',
    'NCD_BMI_MINUS2C': 'Child/Adolescent'
})

print(df)


# 2. CI_Width: Calculate the confidence interval width.**bold text**
*Formula: CI_Width = High - Low

In [ ]:
import pandas as pd

# Example dataset
df = pd.DataFrame({
    'High': [30.5, 25.2, 19.8, 22.4],
    'Low': [28.1, 23.7, 18.6, 21.0]
})

# ✅ Calculate CI_Width
df['CI_Width'] = df['High'] - df['Low']

print(df)


# 3. obesity_level (for the obesity table only):
*Categorize obesity levels based on Mean_Estimate:
   *>= 30 → High


In [ ]:
import pandas as pd

# Example obesity dataset
df_obesity = pd.DataFrame({
    'Mean_Estimate': [32.5, 29.1, 18.4, 34.2, 27.9]
})

# ✅ Categorize obesity levels
df_obesity['obesity_level'] = df_obesity['Mean_Estimate'].apply(
    lambda x: 'High' if x >= 30 else 'Below 30'
)

print(df_obesity)


In [ ]:
def categorize_obesity(x):
    if x >= 30:
        return 'High'
    elif 25 <= x < 30:
        return 'Moderate'
    else:
        return 'Low'

df_obesity['obesity_level'] = df_obesity['Mean_Estimate'].apply(categorize_obesity)


# * 25–29.9 → Moderate

In [ ]:
import pandas as pd

# Example obesity dataset
df_obesity = pd.DataFrame({
    'Mean_Estimate': [32.5, 29.1, 24.7, 34.2, 18.9]
})

# ✅ Categorize obesity levels based on Mean_Estimate
def categorize_obesity(x):
    if x >= 30:
        return 'High'
    elif 25 <= x < 30:
        return 'Moderate'
    else:
        return 'Low'

df_obesity['obesity_level'] = df_obesity['Mean_Estimate'].apply(categorize_obesity)

print(df_obesity)


< 25 → Low

In [ ]:
import pandas as pd

# Example obesity dataset
df_obesity = pd.DataFrame({
    'Mean_Estimate': [32.5, 29.1, 24.7, 34.2, 18.9]
})

# ✅ Categorize obesity levels based on Mean_Estimate
def categorize_obesity(x):
    if x >= 30:
        return 'High'
    elif 25 <= x < 30:
        return 'Moderate'
    elif x < 25:
        return 'Low'

df_obesity['obesity_level'] = df_obesity['Mean_Estimate'].apply(categorize_obesity)

print(df_obesity)


# Each dataset will end up with 10 columns after cleaning and feature engineering

In [ ]:
import pandas as pd

# Example malnutrition dataset
df = pd.DataFrame({
    'ParentLocation': ['India', 'Nepal', 'Bangladesh', 'Sri Lanka'],
    'SpatialDim': ['South Asia']*4,
    'TimeDim': [2018, 2019, 2020, 2021],
    'Dim1': ['Stunting', 'Wasting', 'Underweight', 'Stunting'],
    'Mean_Estimate': [22.5, 18.4, 12.9, 25.7],
    'Low': [20.0, 15.0, 10.0, 22.0],
    'High': [25.0, 22.0, 15.0, 28.0],
    'NumericValue': [22.5, 18.4, 12.9, 25.7],
    'age_group': ['Child/Adolescent']*4
})

# Select & rename columns for consistency
df = df.rename(columns={
    "ParentLocation": "Country",
    "Dim1": "Indicator",
    "TimeDim": "Year",
    "Low": "LowerBound",
    "High": "UpperBound",
    "NumericValue": "Value",
    "SpatialDim": "Region"
})

# Feature engineering
df['CI_Width'] = df['UpperBound'] - df['LowerBound']

# Malnutrition level
df['malnutrition_level'] = pd.cut(
    df['Mean_Estimate'],
    bins=[-float('inf'), 10, 19.9, float('inf')],
    labels=['Low', 'Moderate', 'High']
)

# Keep only 10 columns
df = df[['Country', 'Region', 'Year', 'Indicator', 'Mean_Estimate',
         'LowerBound', 'UpperBound', 'Value', 'age_group', 'CI_Width']]

print(df)


Column Descriptions


In [ ]:
import pandas as pd
import pycountry

# Example raw dataset
df = pd.DataFrame({
    'Dataset': ['NCD_BMI_30C', 'NCD_BMI_18C', 'NCD_BMI_PLUS2C', 'NCD_BMI_MINUS2C'],
    'Year': [2015, 2016, 2017, 2018],
    'Gender': ['Male', 'Female', 'Both', 'Male'],
    'Mean_Estimate': [32.5, 22.7, 19.4, 17.8],
    'Low': [30.0, 21.0, 18.0, 16.0],
    'High': [35.0, 24.0, 21.0, 19.0],
    'Country_Code': ['IND', 'USA', 'NPL', 'AFR']  # 3-letter codes
})

# 1️⃣ Assign Age_Group
df['Age_Group'] = df['Dataset'].map({
    'NCD_BMI_30C': 'Adult',
    'NCD_BMI_18C': 'Adult',
    'NCD_BMI_PLUS2C': 'Child/Adolescent',
    'NCD_BMI_MINUS2C': 'Child/Adolescent'
})

# 2️⃣ Convert Country code to full name and WHO Region
special_regions = {
    'GLOBAL': 'Global',
    'AFR': 'Africa',
    'AMR': 'Americas',
    'EMR': 'Eastern Mediterranean Region',
    'EUR': 'Europe',
    'SEAR': 'South-East Asia Region',
    'WPR': 'Western Pacific Region'
}

def convert_country(code):
    code = str(code).upper()
    if code in special_regions:
        return special_regions[code]
    country = pycountry.countries.get(alpha_3=code)
    return country.name if country else 'Unmatched'

def get_region(code):
    code = str(code).upper()
    return special_regions.get(code, 'Unknown')  # WHO region or 'Unknown'

df['Country'] = df['Country_Code'].apply(convert_country)
df['Region'] = df['Country_Code'].apply(get_region)

# 3️⃣ CI_Width
df['CI_Width'] = df['High'] - df['Low']

# 4️⃣ Obesity_Level (only for obesity datasets)
def categorize_obesity(x):
    if x >= 30:
        return 'High'
    elif 25 <= x < 30:
        return 'Moderate'
    else:
        return 'Low'

# Apply only if this is an obesity dataset
df['Obesity_Level'] = df['Mean_Estimate'].apply(categorize_obesity)

# 5️⃣ Placeholder for Malnutrition_Level (if processing malnutrition table separately)
df['Malnutrition_Level'] = None  # Fill this when handling malnutrition data

# 6️⃣ Select/Order Columns according to your schema
df = df[['Year', 'Gender', 'Mean_Estimate', 'Low', 'High', 'Age_Group',
         'Country', 'Region', 'CI_Width', 'Obesity_Level', 'Malnutrition_Level']]

print(df)

### **For Malnutrition:**

In [ ]:
url = "https://ghoapi.azureedge.net/api/NCD_BMI_18C"    #Underweight in adults (BMI < 18.5)

import requests

response = requests.get(url)

data = response.json()

data

In [ ]:
data.keys()

In [ ]:
import pandas as pd
df3 = pd.DataFrame(data['value'])
df3.head()

In [ ]:
df3.shape

In [ ]:
df3["age_group"] ="Underweight in adults"

In [ ]:
df3.head()

In [ ]:
df3.columns

### Thinness in children

In [ ]:
url = "https://ghoapi.azureedge.net/api/NCD_BMI_MINUS2C"

import requests

response = requests.get(url)

data = response.json()

data

In [ ]:
data.keys()

In [ ]:
import pandas as pd
df4 = pd.DataFrame(data['value'])
df4.head()

In [ ]:
df4.shape

In [ ]:
df4["age_group"] ="Thinness in children"

In [ ]:
df4.head()

In [ ]:
df4.columns

### Combine the two malnutrition datasets into one dataframe called df_malnutrition.

In [ ]:
import pandas as pd

# Combine them into one dataframe
df_malnutrition = pd.concat([df3, df4], ignore_index=True)

print(df_malnutrition.head())


### Filter each dataset to include only records from the years 2012 to 2022

In [ ]:
# Filter df3 for years 2012–2022
df3_filtered = df3[(df3['TimeDim'] >= 2012) & (df3['TimeDim'] <= 2022)]

# Filter df4 for years 2012–2022
df4_filtered = df4[(df4['TimeDim'] >= 2012) & (df4['TimeDim'] <= 2022)]


In [ ]:
import pandas as pd

# Combine the filtered datasets
df_obesity = pd.concat([df3_filtered, df4_filtered], ignore_index=True)

# Optional: check
print(df_obesity['TimeDim'].min(), df_obesity['TimeDim'].max())  # Should show 2012 and 2022
print(df_obesity.shape)  # Number of rows and columns


### Step 2: 🧹 Data Cleaning & Feature Engineering

# Columns to Retain:

In [ ]:
import pandas as pd

columns_needed = [
    "ParentLocation",
    "Dim1",
    "TimeDim",
    "Low",
    "High",
    "NumericValue",
    "SpatialDim",
    "age_group"
]

df2_filtered = df2[columns_needed]

print(df2_filtered.head())


# Rename Columns for Consistency:

In [ ]:
# Rename columns for consistency
df2_filtered = df2_filtered.rename(columns={
    "ParentLocation": "Country",
    "Dim1": "Indicator",
    "TimeDim": "Year",
    "Low": "LowerBound",
    "High": "UpperBound",
    "NumericValue": "Value",
    "SpatialDim": "Region"
})

print(df2_filtered.head())


# Convert Country Codes to Full Names using pycountry:

# 1. Step-by-step instructions:
* Install ‘pycountry’ package

In [ ]:
!pip install pycountry

# Import the package in your Python script or notebook

In [ ]:
import pycountry

special_cases = {
    'GLOBAL': 'Global',
    'WB_LMI': 'Low & Middle Income',
    'WB_HI': 'High Income',
    'WB_LI': 'Low Income',
    'EMR': 'Eastern Mediterranean Region',
    'EUR': 'Europe',
    'AFR': 'Africa',
    'SEAR': 'South-East Asia Region',
    'WPR': 'Western Pacific Region',
    'AMR': 'Americas Region',
    'WB_UMI': 'Upper Middle Income'
}

def convert_country(code):
    # Handle special cases first
    if code in special_cases:
        return special_cases[code]

    # Try to look up in pycountry using Alpha-3 code
    try:
        return pycountry.countries.get(alpha_3=code).name
    except AttributeError:
        # If Alpha-3 fails, try Alpha-2
        try:
            return pycountry.countries.get(alpha_2=code).name
        except AttributeError:
            return code  # If no match found, keep original

# Define a function to convert the 3-letter codes to full names

In [ ]:
import pycountry
print(convert_country('AFR'))

# Apply the function to the Country colum

In [ ]:
import pandas as pd
import pycountry

# Example data
df = pd.DataFrame({
    'Country': ['IND', 'USA', 'AFR', 'FRA', 'CHN']
})

# Function to convert 3-letter codes to full names (handles regions too)
def convert_country(code):
    regions = {
        'AFR': 'Africa',
        'EUR': 'Europe',
        'ASI': 'Asia',
        'AME': 'America',
        'OCE': 'Oceania'
    }
    code = str(code).upper()

    if code in regions:
        return regions[code]

    country = pycountry.countries.get(alpha_3=code)
    return country.name if country else "Invalid country code"

# ✅ Apply the function to the Country column
df['Country_Full'] = df['Country'].apply(convert_country)

print(df)


Handle any remaining unmatched codes using the following dictionary for special values like 'GLOBAL', 'AFR', 'SEAR', etc.

In [ ]:
import pandas as pd
import pycountry

# Example DataFrame
df = pd.DataFrame({
    'Country': ['IND', 'USA', 'AFR', 'GLOBAL', 'SEAR', 'EUR', 'XYZ']
})

# Dictionary for special or regional codes
special_regions = {
    'GLOBAL': 'Global',
    'AFR': 'Africa',
    'AMR': 'Americas',
    'EMR': 'Eastern Mediterranean Region',
    'EUR': 'Europe',
    'SEAR': 'South-East Asia Region',
    'WPR': 'Western Pacific Region'
}

# Function to convert country/region codes to full names
def convert_country(code):
    if not code:
        return "Unknown"

    code = str(code).upper()

    # 1️⃣ Check special codes
    if code in special_regions:
        return special_regions[code]

    # 2️⃣ Try pycountry for standard ISO 3-letter country codes
    country = pycountry.countries.get(alpha_3=code)
    if country:
        return country.name

    # 3️⃣ Fallback for unmatched codes
    return "Unmatched"

# ✅ Apply the function
df['Country_Full'] = df['Country'].apply(convert_country)

print(df)

## New Columns to Create:
1. age_group: Manually assign this column based on the dataset source.
* Use "Adult" for datasets NCD_BMI_30C and NCD_BMI_18C.

In [ ]:
import pandas as pd

# Example DataFrames (already loaded separately)
df_30C = pd.DataFrame({'Dataset': ['NCD_BMI_30C'] * 3, 'Value': [25, 27, 29]})
df_18C = pd.DataFrame({'Dataset': ['NCD_BMI_18C'] * 3, 'Value': [20, 22, 23]})

# ✅ Assign manually based on dataset
df_30C['age_group'] = 'Adult'
df_18C['age_group'] = 'Adult'

# Combine if needed
df_combined = pd.concat([df_30C, df_18C], ignore_index=True)
print(df_combined)


# *Use "Child/Adolescent" for datasets NCD_BMI_PLUS2C and NCD_BMI_MINUS2C.

In [ ]:
import pandas as pd

# Example combined dataset
df = pd.DataFrame({
    'Dataset': ['NCD_BMI_30C', 'NCD_BMI_18C', 'NCD_BMI_PLUS2C', 'NCD_BMI_MINUS2C'],
    'Value': [25.3, 22.7, 19.4, 17.8]
})

# ✅ Assign age group based on dataset name
df['age_group'] = df['Dataset'].map({
    'NCD_BMI_30C': 'Adult',
    'NCD_BMI_18C': 'Adult',
    'NCD_BMI_PLUS2C': 'Child/Adolescent',
    'NCD_BMI_MINUS2C': 'Child/Adolescent'
})

print(df)


# 2. CI_Width: Calculate the confidence interval width.
* Formula: CI_Width = High - Low


In [ ]:
import pandas as pd

# Example dataset
df = pd.DataFrame({
    'High': [30.5, 25.2, 19.8, 22.4],
    'Low': [28.1, 23.7, 18.6, 21.0]
})

# ✅ Calculate CI_Width
df['CI_Width'] = df['High'] - df['Low']

print(df)


# 3. malnutrition_level (for the malnutrition table only):
*Categorize malnutrition levels based on Mean_Estimate:
   *>= 20 → High

In [ ]:
import pandas as pd

# Example dataset
df = pd.DataFrame({
    'High': [30.5, 25.2, 19.8, 22.4],
    'Low': [28.1, 23.7, 18.6, 21.0],
    'Mean_Estimate': [22.5, 18.4, 12.9, 25.7]  # Example mean values
})

# ✅ Calculate CI_Width
df['CI_Width'] = df['High'] - df['Low']

# ✅ Categorize malnutrition_level based on Mean_Estimate
df['malnutrition_level'] = pd.cut(
    df['Mean_Estimate'],
    bins=[-float('inf'), 10, 20, float('inf')],
    labels=['Low', 'Moderate', 'High']
)

print(df)


* 10–19.9 → Moderate

In [ ]:
import pandas as pd

# Example dataset
df = pd.DataFrame({
    'High': [30.5, 25.2, 19.8, 22.4],
    'Low': [28.1, 23.7, 18.6, 21.0],
    'Mean_Estimate': [22.5, 18.4, 12.9, 25.7]  # Example mean values
})

# ✅ Calculate CI_Width
df['CI_Width'] = df['High'] - df['Low']

# ✅ Categorize malnutrition_level based on Mean_Estimate
df['malnutrition_level'] = pd.cut(
    df['Mean_Estimate'],
    bins=[-float('inf'), 9.999, 19.9, float('inf')],
    labels=['Low', 'Moderate', 'High']
)

print(df)


* < 10 → Low

# Each dataset will end up with 10 columns after cleaning and feature engineering

In [ ]:
import pandas as pd

# Example dataset
df = pd.DataFrame({
    'High': [30.5, 25.2, 19.8, 22.4],
    'Low': [28.1, 23.7, 18.6, 21.0],
    'Mean_Estimate': [22.5, 18.4, 12.9, 25.7]  # Example mean values
})

# Calculate CI_Width
df['CI_Width'] = df['High'] - df['Low']

# Categorize malnutrition_level
df['malnutrition_level'] = pd.cut(
    df['Mean_Estimate'],
    bins=[-float('inf'), 10, 19.9, float('inf')],
    labels=['Low', 'Moderate', 'High']
)

print(df)


In [ ]:
import pandas as pd

# Example malnutrition dataset
df = pd.DataFrame({
    'ParentLocation': ['India', 'Nepal', 'Bangladesh', 'Sri Lanka'],
    'SpatialDim': ['South Asia']*4,
    'TimeDim': [2018, 2019, 2020, 2021],
    'Dim1': ['Stunting', 'Wasting', 'Underweight', 'Stunting'],
    'Mean_Estimate': [22.5, 18.4, 12.9, 25.7],
    'Low': [20.0, 15.0, 10.0, 22.0],
    'High': [25.0, 22.0, 15.0, 28.0],
    'NumericValue': [22.5, 18.4, 12.9, 25.7],
    'age_group': ['Child/Adolescent']*4
})

# Select & rename columns for consistency
df = df.rename(columns={
    "ParentLocation": "Country",
    "Dim1": "Indicator",
    "TimeDim": "Year",
    "Low": "LowerBound",
    "High": "UpperBound",
    "NumericValue": "Value",
    "SpatialDim": "Region"
})

# Feature engineering
df['CI_Width'] = df['UpperBound'] - df['LowerBound']

# Malnutrition level
df['malnutrition_level'] = pd.cut(
    df['Mean_Estimate'],
    bins=[-float('inf'), 10, 19.9, float('inf')],
    labels=['Low', 'Moderate', 'High']
)

# Keep only 10 columns
df = df[['Country', 'Region', 'Year', 'Indicator', 'Mean_Estimate',
         'LowerBound', 'UpperBound', 'Value', 'age_group', 'CI_Width']]

print(df)


### Column Descriptions

In [ ]:
import pandas as pd
import pycountry

# Example raw dataset
df = pd.DataFrame({
    'Dataset': ['NCD_BMI_30C', 'NCD_BMI_18C', 'NCD_BMI_PLUS2C', 'NCD_BMI_MINUS2C'],
    'Year': [2015, 2016, 2017, 2018],
    'Gender': ['Male', 'Female', 'Both', 'Male'],
    'Mean_Estimate': [32.5, 22.7, 19.4, 17.8],
    'Low': [30.0, 21.0, 18.0, 16.0],
    'High': [35.0, 24.0, 21.0, 19.0],
    'Country_Code': ['IND', 'USA', 'NPL', 'AFR']  # 3-letter codes
})

# 1️⃣ Assign Age_Group
df['Age_Group'] = df['Dataset'].map({
    'NCD_BMI_30C': 'Adult',
    'NCD_BMI_18C': 'Adult',
    'NCD_BMI_PLUS2C': 'Child/Adolescent',
    'NCD_BMI_MINUS2C': 'Child/Adolescent'
})

# 2️⃣ Convert Country code to full name and WHO Region
special_regions = {
    'GLOBAL': 'Global',
    'AFR': 'Africa',
    'AMR': 'Americas',
    'EMR': 'Eastern Mediterranean Region',
    'EUR': 'Europe',
    'SEAR': 'South-East Asia Region',
    'WPR': 'Western Pacific Region'
}

def convert_country(code):
    code = str(code).upper()
    if code in special_regions:
        return special_regions[code]
    country = pycountry.countries.get(alpha_3=code)
    return country.name if country else 'Unmatched'

def get_region(code):
    code = str(code).upper()
    return special_regions.get(code, 'Unknown')  # WHO region or 'Unknown'

df['Country'] = df['Country_Code'].apply(convert_country)
df['Region'] = df['Country_Code'].apply(get_region)

# 3️⃣ CI_Width
df['CI_Width'] = df['High'] - df['Low']

# 4️⃣ Obesity_Level (only for obesity datasets)
def categorize_obesity(x):
    if x >= 30:
        return 'High'
    elif 25 <= x < 30:
        return 'Moderate'
    else:
        return 'Low'

# Apply only if this is an obesity dataset
df['Obesity_Level'] = df['Mean_Estimate'].apply(categorize_obesity)

# 5️⃣ Placeholder for Malnutrition_Level (if processing malnutrition table separately)
df['Malnutrition_Level'] = None  # Fill this when handling malnutrition data

# 6️⃣ Select/Order Columns according to your schema
df = df[['Year', 'Gender', 'Mean_Estimate', 'Low', 'High', 'Age_Group',
         'Country', 'Region', 'CI_Width', 'Obesity_Level', 'Malnutrition_Level']]

print(df)

### Step:3 🧮 Exploratory Data Analysis (EDA) with Python

* Understanding the shape and structure of the data

In [ ]:
# Display first few rows
print(df.head())

# Check shape of dataset
print(df.shape)

# Summary of column types and missing values
print(df.info())

# Basic statistics for numeric columns
print(df.describe())


* Identifying missing values or unusual values

In [ ]:
# Count missing values in each column
print(df.isnull().sum())

# Optional: visualize missing data
import seaborn as sns
import matplotlib.pyplot as plt

sns.heatmap(df.isnull(), cbar=False, cmap="viridis")
plt.title("Missing Values Heatmap")
plt.show()


# Checking distribution of Mean_Estimate, CI_Width, etc.

In [ ]:
# Histogram of Mean_Estimate
sns.histplot(df['Mean_Estimate'], bins=10, kde=True)
plt.title("Distribution of Mean_Estimate")
plt.show()

# Distribution of CI_Width
sns.histplot(df['CI_Width'], bins=10, kde=True)
plt.title("Distribution of CI_Width")
plt.show()


# *Analyzing trends across years, regions, or demographic groups

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Average Mean_Estimate per year
yearly_trend = df.groupby('Year')['Mean_Estimate'].mean().reset_index()

# Line plot
plt.figure(figsize=(8,5))
sns.lineplot(x='Year', y='Mean_Estimate', data=yearly_trend, marker='o')
plt.title('Average Mean_Estimate Over Years')
plt.ylabel('Mean_Estimate (%)')
plt.show()


In [ ]:
sns.lineplot(x='Year', y='Mean_Estimate', hue='Age_Group', data=df, marker='o')
plt.title('Mean_Estimate Trends by Age Group')
plt.show()

### REGIONS

In [ ]:
# Average Mean_Estimate per region
region_trend = df.groupby('Region')['Mean_Estimate'].mean().sort_values()

# Bar plot
plt.figure(figsize=(10,5))
sns.barplot(x=region_trend.index, y=region_trend.values)
plt.xticks(rotation=45)
plt.title('Average Mean_Estimate by Region')
plt.ylabel('Mean_Estimate (%)')
plt.show()


In [ ]:
region_year_trend = df.groupby(['Region','Year'])['Mean_Estimate'].mean().reset_index()

plt.figure(figsize=(10,6))
sns.lineplot(x='Year', y='Mean_Estimate', hue='Region', data=region_year_trend, marker='o')
plt.title('Mean_Estimate Trends by Region Over Years')
plt.show()

### demographic groups

In [ ]:
# Average Mean_Estimate by age_group
age_trend = df.groupby('Age_Group')['Mean_Estimate'].mean()

sns.barplot(x=age_trend.index, y=age_trend.values)
plt.title('Average Mean_Estimate by Age Group')
plt.ylabel('Mean_Estimate (%)')
plt.show()

# If malnutrition_level exists
if 'malnutrition_level' in df.columns:
    malnutrition_trend = df.groupby('malnutrition_level')['Mean_Estimate'].mean()
    sns.barplot(x=malnutrition_trend.index, y=malnutrition_trend.values, palette='viridis')
    plt.title('Mean_Estimate by Malnutrition Level')
    plt.ylabel('Mean_Estimate (%)')
    plt.show()

Comparing obesity vs malnutrition distributions

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Add a label column
df_obesity['Type'] = 'Obesity'
df_malnutrition['Type'] = 'Malnutrition'

# Select relevant columns and rename before concatenating
columns_needed = ['TimeDim', 'NumericValue', 'age_group', 'ParentLocation', 'Type']
df_obesity_cleaned = df_obesity[columns_needed].copy()
df_malnutrition_cleaned = df_malnutrition[columns_needed].copy()

df_obesity_cleaned = df_obesity_cleaned.rename(columns={'TimeDim': 'Year', 'NumericValue': 'Mean_Estimate', 'ParentLocation': 'Region'})
df_malnutrition_cleaned = df_malnutrition_cleaned.rename(columns={'TimeDim': 'Year', 'NumericValue': 'Mean_Estimate', 'ParentLocation': 'Region'})

# Combine them
df_combined = pd.concat([df_obesity_cleaned, df_malnutrition_cleaned], ignore_index=True)

# Distribution Comparison: Histograms / KDE

In [ ]:
plt.figure(figsize=(10,6))
sns.histplot(
    data=df_combined,
    x='Mean_Estimate',
    hue='Type',
    kde=True,            # adds KDE curve
    stat='density',      # normalize
    common_norm=False,   # normalize separately
    element='step',
    alpha=0.5
)

plt.title('Distribution Comparison: Obesity vs Malnutrition')
plt.xlabel('Mean Estimate (%)')
plt.ylabel('Density')
plt.legend(title='Type')
plt.show()

### Boxplot Comparison

In [ ]:
plt.figure(figsize=(8,5))
sns.boxplot(x='Type', y='Mean_Estimate', data=df_combined, palette=['#FF6347','#4682B4'])
plt.title('Boxplot of Mean_Estimate by Type')
plt.ylabel('Mean_Estimate (%)')
plt.show()

### Regional Comparison

In [ ]:
plt.figure(figsize=(12,6))
sns.barplot(x='Region', y='Mean_Estimate', hue='Type', data=df_combined, errorbar=None)
plt.xticks(rotation=45)
plt.title('Average Mean_Estimate by Region: Obesity vs Malnutrition')
plt.ylabel('Mean_Estimate (%)')
plt.show()

Try different visualizations using Matplotlib or Seaborn or Plotly to uncover interesting patterns
* Line plots to explore trends over time

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Average Mean_Estimate over years
yearly_trend = df_combined.groupby('Year')['Mean_Estimate'].mean().reset_index()

plt.figure(figsize=(8,5))
sns.lineplot(x='Year', y='Mean_Estimate', data=yearly_trend, marker='o')
plt.title('Average Mean_Estimate Over Years')
plt.ylabel('Mean_Estimate (%)')
plt.show()

# By age group
plt.figure(figsize=(8,5))
sns.lineplot(x='Year', y='Mean_Estimate', hue='age_group', data=df_combined, marker='o')
plt.title('Trends by Age Group')
plt.show()

* Bar charts to compare top/bottom countries or regions

In [ ]:
# Top 5 countries with highest Mean_Estimate
top_countries = df.groupby('Country')['Mean_Estimate'].mean().sort_values(ascending=False).head(5)

plt.figure(figsize=(8,5))
sns.barplot(x=top_countries.index, y=top_countries.values, palette='viridis')
plt.title('Top 5 Countries by Mean_Estimate')
plt.ylabel('Mean_Estimate (%)')
plt.show()

# Bottom 5
bottom_countries = df.groupby('Country')['Mean_Estimate'].mean().sort_values().head(5)
plt.figure(figsize=(8,5))
sns.barplot(x=bottom_countries.index, y=bottom_countries.values, palette='magma')
plt.title('Bottom 5 Countries by Mean_Estimate')
plt.show()

* Box plots to observe variability by region

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(x='Region', y='Mean_Estimate', data=df)
plt.title('Variability by Region')
plt.xticks(rotation=45)
plt.show()

# By Age Group
sns.boxplot(x='Age_Group', y='Mean_Estimate', data=df)
plt.title('Variability by Age Group')
plt.show()

# * Heatmaps or scatter plots to identify patterns and outliers

In [ ]:
# Pivot table for heatmap
heatmap_data = df.pivot_table(values='Mean_Estimate', index='Region', columns='Year', aggfunc='mean')

plt.figure(figsize=(12,6))
sns.heatmap(heatmap_data, annot=True, fmt=".1f", cmap='YlOrRd')
plt.title('Heatmap of Mean_Estimate by Region and Year')
plt.show()

**scatter plots to identify patterns

In [ ]:
plt.figure(figsize=(8,5))
sns.scatterplot(x='CI_Width', y='Mean_Estimate', hue='Region', data=df, s=100)
plt.title('Mean_Estimate vs CI_Width by Region')
plt.show()

** Optional: Plotly Interactive Visuals

In [ ]:
import plotly.express as px

# Interactive line plot over years
fig = px.line(df, x='Year', y='Mean_Estimate', color='Age_Group', markers=True, title='Trends Over Time')
fig.show()

# Interactive scatter
fig2 = px.scatter(df, x='CI_Width', y='Mean_Estimate', color='Region', size='Mean_Estimate',
                  hover_data=['Country'], title='Mean_Estimate vs CI_Width')
fig2.show()

***Step 4: Insert Data into SQL
* SQLite
**1. Import Libraries

In [ ]:
import sqlite3
import pandas as pd

2. Connect to SQLite Database

In [ ]:
# Connect to SQLite (creates 'health_data.db' if not exists)
conn = sqlite3.connect('health_data.db')

# Create a cursor object
cursor = conn.cursor()

3. Create Table (if not exists)

In [ ]:
create_table_query = """
CREATE TABLE IF NOT EXISTS health_data (
    Country TEXT,
    Region TEXT,
    Year INTEGER,
    Indicator TEXT,
    Mean_Estimate REAL,
    LowerBound REAL,
    UpperBound REAL,
    Value REAL,
    age_group TEXT,
    CI_Width REAL
);
"""

cursor.execute(create_table_query)
conn.commit()

4. Insert Data from Pandas DataFrame

In [ ]:
# Assuming df is your cleaned dataframe
df.to_sql('health_data', conn, if_exists='replace', index=False)

print("Data inserted successfully!")

5. Query Data to Verify

In [ ]:
# Simple query to check inserted data
query_result = pd.read_sql("SELECT * FROM health_data LIMIT 5;", conn)
print(query_result)

6. Close Connection

In [ ]:
conn.close()

**Create 2 tables using Python code
*obesity
*malnutrition


In [ ]:
import sqlite3
import pandas as pd

# Example: assuming df_obesity and df_malnutrition are your cleaned 10-column datasets
# df_obesity.head(), df_malnutrition.head() should have columns:
# ['Country', 'Region', 'Year', 'Indicator', 'Mean_Estimate',
#  'LowerBound', 'UpperBound', 'Value', 'age_group', 'CI_Width']

# 1️⃣ Connect to SQLite database
conn = sqlite3.connect('health_data.db')
cursor = conn.cursor()

# 2️⃣ Create obesity table
cursor.execute("""
CREATE TABLE IF NOT EXISTS obesity (
    Country TEXT,
    Region TEXT,
    Year INTEGER,
    Indicator TEXT,
    Mean_Estimate REAL,
    LowerBound REAL,
    UpperBound REAL,
    Value REAL,
    age_group TEXT,
    CI_Width REAL
);
""")

# 3️⃣ Create malnutrition table
cursor.execute("""
CREATE TABLE IF NOT EXISTS malnutrition (
    Country TEXT,
    Region TEXT,
    Year INTEGER,
    Indicator TEXT,
    Mean_Estimate REAL,
    LowerBound REAL,
    UpperBound REAL,
    Value REAL,
    age_group TEXT,
    CI_Width REAL
);
""")

conn.commit()

# 4️⃣ Insert data into obesity table
df_obesity.to_sql('obesity', conn, if_exists='replace', index=False)

# 5️⃣ Insert data into malnutrition table
df_malnutrition.to_sql('malnutrition', conn, if_exists='replace', index=False)

print("Both tables created and data inserted successfully!")

# 6️⃣ Optional: verify
print(pd.read_sql("SELECT * FROM obesity LIMIT 5;", conn))
print(pd.read_sql("SELECT * FROM malnutrition LIMIT 5;", conn))

# 7️⃣ Close connection
conn.close()

5. Insert Cleaned Data
*Insert rows from your final DataFrames (df_obesity and df_malnutrition) into the respective tables using .iterrows() in a for loop.


In [ ]:
import sqlite3
import pandas as pd
import pycountry

# Example: assuming df_obesity and df_malnutrition are your cleaned 10-column datasets
# df_obesity.head(), df_malnutrition.head() should have columns:
# ['Country', 'Region', 'Year', 'Indicator', 'Mean_Estimate',
#  'LowerBound', 'UpperBound', 'Value', 'age_group', 'CI_Width']

# 1️⃣ Connect to SQLite database
conn = sqlite3.connect('health_data.db')
cursor = conn.cursor()

# 2️⃣ Create obesity table
cursor.execute("""
CREATE TABLE IF NOT EXISTS obesity (
    Country TEXT,
    Region TEXT,
    Year INTEGER,
    Gender TEXT,
    Mean_Estimate REAL,
    LowerBound REAL,
    UpperBound REAL,
    Value REAL,
    age_group TEXT,
    CI_Width REAL,
    obesity_level TEXT
);
""")

# 3️⃣ Create malnutrition table
cursor.execute("""
CREATE TABLE IF NOT EXISTS malnutrition (
    Country TEXT,
    Region TEXT,
    Year INTEGER,
    Indicator TEXT,
    Mean_Estimate REAL,
    LowerBound REAL,
    UpperBound REAL,
    Value REAL,
    age_group TEXT,
    CI_Width REAL,
    malnutrition_level TEXT
);
""")

conn.commit()


# Dictionary for special or regional codes
special_regions = {
    'GLOBAL': 'Global',
    'AFR': 'Africa',
    'AMR': 'Americas',
    'EMR': 'Eastern Mediterranean Region',
    'EUR': 'Europe',
    'SEAR': 'South-East Asia Region',
    'WPR': 'Western Pacific Region',
    'WB_LMI': 'Low & Middle Income',
    'WB_HI': 'High Income',
    'WB_LI': 'Low Income',
    'WB_UMI': 'Upper Middle Income'
}

def convert_country(code):
    if not code:
        return "Unknown"

    code = str(code).upper()

    # 1️⃣ Check special codes
    if code in special_regions:
        return special_regions[code]

    # 2️⃣ Try pycountry for standard ISO 3-letter country codes
    try:
        country = pycountry.countries.get(alpha_3=code)
        if country:
            return country.name
    except AttributeError:
        pass # Ignore error if code is not alpha_3

    # 3️⃣ Fallback for unmatched codes
    return "Unmatched"

def get_region(code):
    code = str(code).upper()
    return special_regions.get(code, 'Unknown')  # WHO region or 'Unknown'


# Apply cleaning and feature engineering to df_obesity
df_obesity = df_obesity.rename(columns={
    "TimeDim": "Year",
    "Dim1": "Gender",
    "NumericValue": "Mean_Estimate",
    "Low": "LowerBound",
    "High": "UpperBound",
    "ParentLocation": "Region",
    "SpatialDim": "Country"
})

df_obesity["Country"] = df_obesity["Country"].apply(convert_country)
df_obesity['Region'] = df_obesity['ParentLocationCode'].apply(get_region)
df_obesity['CI_Width'] = df_obesity['UpperBound'] - df_obesity['LowerBound']

def categorize_obesity(x):
    if x >= 30:
        return 'High'
    elif 25 <= x < 30:
        return 'Moderate'
    else:
        return 'Low'

df_obesity['obesity_level'] = df_obesity['Mean_Estimate'].apply(categorize_obesity)


# Apply cleaning and feature engineering to df_malnutrition
df_malnutrition = df_malnutrition.rename(columns={
    "TimeDim": "Year",
    "Dim1": "Indicator",
    "NumericValue": "Mean_Estimate",
    "Low": "LowerBound",
    "High": "UpperBound",
    "ParentLocation": "Region",
    "SpatialDim": "Country"
})

df_malnutrition["Country"] = df_malnutrition["Country"].apply(convert_country)
df_malnutrition['Region'] = df_malnutrition['ParentLocationCode'].apply(get_region)
df_malnutrition['CI_Width'] = df_malnutrition['UpperBound'] - df_malnutrition['LowerBound']

def categorize_malnutrition(x):
    if x >= 20:
        return 'High'
    elif 10 <= x < 20:
        return 'Moderate'
    else:
        return 'Low'

df_malnutrition['malnutrition_level'] = df_malnutrition['Mean_Estimate'].apply(categorize_malnutrition)


# Select only the columns needed for the database
obesity_columns = ['Country', 'Region', 'Year', 'Gender', 'Mean_Estimate',
                   'LowerBound', 'UpperBound', 'Value', 'age_group', 'CI_Width', 'obesity_level']
malnutrition_columns = ['Country', 'Region', 'Year', 'Indicator', 'Mean_Estimate',
                        'LowerBound', 'UpperBound', 'Value', 'age_group', 'CI_Width', 'malnutrition_level']

df_obesity = df_obesity[obesity_columns]
df_malnutrition = df_malnutrition[malnutrition_columns]


# 4️⃣ Insert data into obesity table
df_obesity.to_sql('obesity', conn, if_exists='replace', index=False)

# 5️⃣ Insert data into malnutrition table
df_malnutrition.to_sql('malnutrition', conn, if_exists='replace', index=False)

print("✅ Both tables created and data inserted successfully using to_sql()!")

# 6️⃣ Optional: verify
print(pd.read_sql("SELECT * FROM obesity LIMIT 5;", conn))
print(pd.read_sql("SELECT * FROM malnutrition LIMIT 5;", conn))

# 7️⃣ Close connection
conn.close()

### Step 4: Query Writing Streamlit
🧋 Obesity Table (10 Queries)

In [ ]:
!pip install streamlit
!streamlit run app.py

# 1. Top 5 regions with the highest average obesity levels in the most recent year(2022)

In [ ]:
import sqlite3
import pandas as pd
import streamlit as st

# Connect to SQLite database
conn = sqlite3.connect('health_data.db')

# SQL query to find top 5 regions
query = """
SELECT Region, AVG(Mean_Estimate) AS Avg_Obesity
FROM obesity
WHERE Year = 2022
GROUP BY Region
ORDER BY Avg_Obesity DESC
LIMIT 5;
"""

# Execute query and show results
top5_regions = pd.read_sql(query, conn)

st.write("🌍 **Top 5 Regions with Highest Average Obesity Levels in 2022**")
st.dataframe(top5_regions)
st.bar_chart(top5_regions.set_index("Region"))

2. Top 5 countries with highest obesity estimates

In [ ]:
conn = sqlite3.connect('health_data.db')
query = """
SELECT
    Country,
    ROUND(AVG(Mean_Estimate), 2) AS Avg_Obesity_Level
FROM obesity
GROUP BY Country
ORDER BY Avg_Obesity_Level DESC
LIMIT 5;
"""

top5_countries = pd.read_sql(query, conn)

# Display results
print("🌍 Top 5 Countries with Highest Average Obesity Estimates:")
print(top5_countries)

conn.close()

3. Obesity trend in India over the years(Mean_estimate)

In [ ]:
# Connect to the database
conn = sqlite3.connect('health_data.db')

# Run query
query = """
SELECT
    Year,
    ROUND(AVG(Mean_Estimate), 2) AS Avg_Obesity_Level
FROM obesity
WHERE Country = 'India'
GROUP BY Year
ORDER BY Year ASC;
"""

india_trend = pd.read_sql(query, conn)

# Display data
print("📊 Obesity Trend in India Over the Years:")
print(india_trend)

# Plot tr

4. Average obesity by gender

In [ ]:
# Connect to SQLite
conn = sqlite3.connect('health_data.db')

# Run query
query = """
SELECT
    Gender,
    ROUND(AVG(Mean_Estimate), 2) AS Avg_Obesity_Level
FROM obesity
GROUP BY Gender
ORDER BY Avg_Obesity_Level DESC;
"""

gender_obesity = pd.read_sql(query, conn)

# Display table
print("⚖️ Average Obesity by Gender:")
print(gender_obesity)

# Plot bar chart
plt.figure(figsize=(6,4))
plt.bar(gender_obesity['Gender'], gender_obesity['Avg_Obesity_Level'], color='skyblue')
plt.title('Average Obesity by Gender', fontsize=14)
plt.xlabel('Gender')
plt.ylabel('Average Obesity Level (%)')
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.show()

conn.close()

5. Country count by obesity level category and age group


In [ ]:
# Connect to DB
conn = sqlite3.connect('health_data.db')

query = """
WITH categorized AS (
    SELECT
        Country,
        age_group,
        CASE
            WHEN Mean_Estimate < 10 THEN 'Low'
            WHEN Mean_Estimate BETWEEN 10 AND 20 THEN 'Moderate'
            WHEN Mean_Estimate BETWEEN 20 AND 30 THEN 'High'
            ELSE 'Very High'
        END AS Obesity_Level_Category
    FROM obesity
    WHERE Year = 2022
)
SELECT
    Obesity_Level_Category,
    age_group,
    COUNT(DISTINCT Country) AS Country_Count
FROM categorized
GROUP BY Obesity_Level_Category, age_group
ORDER BY
    CASE
        WHEN Obesity_Level_Category = 'Low' THEN 1
        WHEN Obesity_Level_Category = 'Moderate' THEN 2
        WHEN Obesity_Level_Category = 'High' THEN 3
        WHEN Obesity_Level_Category = 'Very High' THEN 4
    END,
    age_group;
"""

obesity_category = pd.read_sql(query, conn)

print("🏳️ Country Count by Obesity Level Category and Age Group (2022):")
print(obesity_category)

# Visualize
plt.figure(figsize=(8,5))
for group in obesity_category['age_group'].unique():
    subset = obesity_category[obesity_category['age_group'] == group]
    plt.bar(subset['Obesity_Level_Category'], subset['Country_Count'], label=group)

plt.title('Country Count by Obesity Level Category and Age Group (2022)')
plt.xlabel('Obesity Level Category')
plt.ylabel('Number of Countries')
plt.legend(title='Age Group')
plt.grid(axis='y', linestyle='--', alpha=0.6) # Fixed the incomplete line
plt.show()

conn.close()

6. Top 5 countries least reliable countries(with highest CI_Width) and Top 5 most consistent countries (smallest average CI_Width)

In [ ]:
# Connect to the database
conn = sqlite3.connect('health_data.db')

# Top 5 least reliable (highest CI_Width)
query_least_reliable = """
SELECT
    Country,
    ROUND(AVG(CI_Width), 3) AS Avg_CI_Width
FROM obesity
GROUP BY Country
ORDER BY Avg_CI_Width DESC
LIMIT 5;
"""

least_reliable = pd.read_sql(query_least_reliable, conn)

# Top 5 most consistent (lowest CI_Width)
query_most_consistent = """
SELECT
    Country,
    ROUND(AVG(CI_Width), 3) AS Avg_CI_Width
FROM obesity
GROUP BY Country
ORDER BY Avg_CI_Width ASC
LIMIT 5;
"""

most_consistent = pd.read_sql(query_most_consistent, conn)

print("⚠️ Top 5 Least Reliable Countries (Highest Average CI_Width):")
print(least_reliable)
print("\n✅ Top 5 Most Consistent Countries (Lowest Average CI_Width):")
print(most_consistent)

conn.close()

7. Average obesity by age group

In [ ]:
conn = sqlite3.connect('health_data.db')
query = """
SELECT
    Age_Group,
    ROUND(AVG(Mean_Estimate), 2) AS Avg_Obesity_Level
FROM obesity
GROUP BY Age_Group
ORDER BY Avg_Obesity_Level DESC;
"""

age_obesity = pd.read_sql(query, conn)

plt.figure(figsize=(6, 4))
plt.bar(age_obesity['age_group'], age_obesity['Avg_Obesity_Level'], color=['lightcoral', 'skyblue'])
plt.title('Average Obesity by Age Group', fontsize=14)
plt.xlabel('Age Group')
plt.ylabel('Average Obesity Level (%)')
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.show()

# st.dataframe(age_obesity) # This line is commented out to avoid error in Colab environment
conn.close()

8. Top 10 Countries with consistent low obesity (low average + low CI)over the years

In [ ]:
# Connect to DB
conn = sqlite3.connect('health_data.db')

# Query top 10 countries with consistently low obesity & low CI_Width
query = """
SELECT
    Country,
    ROUND(AVG(Mean_Estimate), 2) AS Avg_Obesity_Level,
    ROUND(AVG(CI_Width), 3) AS Avg_CI_Width
FROM obesity
GROUP BY Country
HAVING Avg_Obesity_Level IS NOT NULL
ORDER BY Avg_Obesity_Level ASC, Avg_CI_Width ASC
LIMIT 10;
"""

consistent_low_obesity = pd.read_sql(query, conn)

# Display results
print("🌿 Top 10 Countries with Consistent Low Obesity and High Data Reliability:")
print(consistent_low_obesity)

# Visualization
plt.figure(figsize=(8,5))
plt.scatter(
    consistent_low_obesity['Avg_Obesity_Level'],
    consistent_low_obesity['Avg_CI_Width'],
    color='mediumseagreen',
    s=80
)
for i, row in consistent_low_obesity.iterrows():
    plt.text(row['Avg_Obesity_Level']+0.1, row['Avg_CI_Width']+0.001, row['Country'], fontsize=9)

plt.title('Top 10 Consistently Low-Obesity Countries (Low Mean + Low CI)', fontsize=13)
plt.xlabel('Average Obesity Level (%)')
plt.ylabel('Average CI Width')
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

conn.close()

9. Countries where female obesity exceeds male by large margin (same       year)

In [ ]:
# Connect to database
conn = sqlite3.connect('health_data.db')

query = """
SELECT
    f.Country,
    f.Year,
    ROUND(f.Mean_Estimate, 2) AS Female_Obesity,
    ROUND(m.Mean_Estimate, 2) AS Male_Obesity,
    ROUND(f.Mean_Estimate - m.Mean_Estimate, 2) AS Difference
FROM obesity f
JOIN obesity m
    ON f.Country = m.Country
    AND f.Year = m.Year
WHERE f.Gender LIKE '%FMLE%'
  AND m.Gender LIKE '%MLE%'
  AND (f.Mean_Estimate - m.Mean_Estimate) > 5
ORDER BY Difference DESC
LIMIT 10;
"""

gender_gap = pd.read_sql(query, conn)

print("💃 Countries Where Female Obesity Exceeds Male by Large Margin:")
print(gender_gap)

conn.close()

10. Global average obesity percentage per year

In [ ]:
# Connect to database
conn = sqlite3.connect('health_data.db')

# Query global average obesity per year
query = """
SELECT
    Year,
    ROUND(AVG(Mean_Estimate), 2) AS Global_Avg_Obesity
FROM obesity
GROUP BY Year
ORDER BY Year ASC;
"""

global_trend = pd.read_sql(query, conn)

# Display results
print("🌍 Global Average Obesity Percentage Per Year:")
print(global_trend)

# Plot trend
plt.figure(figsize=(8,5))
plt.plot(global_trend['Year'], global_trend['Global_Avg_Obesity'], marker='o', color='mediumblue')
plt.title('Global Average Obesity Percentage Per Year', fontsize=14)
plt.xlabel('Year')
plt.ylabel('Average Obesity (%)')
plt.grid(True)
plt.show()

conn.close()

### 👾 Malnutrition Table (10 Queries)

1. Avg. malnutrition by age group

In [ ]:
# Connect to database
conn = sqlite3.connect('health_data.db')

# Query average malnutrition by age group
query = """
SELECT
    age_group,
    ROUND(AVG(Mean_Estimate), 2) AS Avg_Malnutrition
FROM malnutrition
GROUP BY age_group
ORDER BY Avg_Malnutrition DESC;
"""

age_malnutrition = pd.read_sql(query, conn)

# Display results
print("👶🧍 Average Malnutrition by Age Group:")
print(age_malnutrition)

# Plot visualization
plt.figure(figsize=(6,4))
plt.bar(age_malnutrition['age_group'], age_malnutrition['Avg_Malnutrition'], color=['orange','skyblue'])
plt.title('Average Malnutrition by Age Group', fontsize=14)
plt.xlabel('Age Group')
plt.ylabel('Average Malnutrition (%)')
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.show()

conn.close()

2. Top 5 countries with highest malnutrition(mean_estimate)

In [ ]:
# Connect to database
conn = sqlite3.connect('health_data.db')

# Query top 5 countries with highest malnutrition
query = """
SELECT
    Country,
    ROUND(AVG(Mean_Estimate), 2) AS Avg_Malnutrition
FROM malnutrition
GROUP BY Country
ORDER BY Avg_Malnutrition DESC
LIMIT 5;
"""

top5_malnutrition = pd.read_sql(query, conn)

print("⚠️ Top 5 Countries with Highest Malnutrition Rates:")
print(top5_malnutrition)

conn.close()

3. Malnutrition trend in African region over the years

In [ ]:
# Connect to database
conn = sqlite3.connect('health_data.db')

# Query African malnutrition trend
query = """
SELECT
    Year,
    ROUND(AVG(Mean_Estimate), 2) AS Avg_Malnutrition
FROM malnutrition
WHERE Region LIKE '%Africa%'
GROUP BY Year
ORDER BY Year ASC;
"""

africa_trend = pd.read_sql(query, conn)

# Display results
print("📊 Malnutrition Trend in African Region Over the Years:")
print(africa_trend)

# Plot trend
plt.figure(figsize=(8,5))
plt.plot(africa_trend['Year'], africa_trend['Avg_Malnutrition'], marker='o', color='orangered')
plt.title('Malnutrition Trend in African Region Over the Years', fontsize=14)
plt.xlabel('Year')
plt.ylabel('Average Malnutrition (%)')
plt.grid(True)
plt.show()

conn.close()

4. Gender-based average malnutrition

In [ ]:
conn = sqlite3.connect('health_data.db')

# Query average malnutrition by gender
query = """
SELECT
    Indicator AS Gender,
    ROUND(AVG(Mean_Estimate), 2) AS Avg_Malnutrition
FROM malnutrition
GROUP BY Indicator
ORDER BY Avg_Malnutrition DESC;
"""

gender_malnutrition = pd.read_sql(query, conn)

# Display results
print("⚖️ Average Malnutrition by Gender:")
print(gender_malnutrition)

# Plot bar chart
plt.figure(figsize=(6,4))
plt.bar(gender_malnutrition['Gender'], gender_malnutrition['Avg_Malnutrition'], color=['orange','skyblue'])
plt.title('Average Malnutrition by Gender', fontsize=14)
plt.xlabel('Gender')
plt.ylabel('Average Malnutrition (%)')
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.show()

conn.close()

5. Malnutrition level-wise (average CI_Width by age group)

In [ ]:
conn = sqlite3.connect('health_data.db')

# Query average CI_Width by age group
query = """
SELECT
    age_group,
    ROUND(AVG(CI_Width), 3) AS Avg_CI_Width
FROM malnutrition
GROUP BY age_group
ORDER BY Avg_CI_Width ASC;
"""

ci_by_age = pd.read_sql(query, conn)

print("📏 Average CI_Width by Age Group (Malnutrition):")
print(ci_by_age)

# Plot
plt.figure(figsize=(6,4))
plt.bar(ci_by_age['age_group'], ci_by_age['Avg_CI_Width'], color=['lightgreen','salmon'])
plt.title('Average CI_Width by Age Group (Malnutrition)', fontsize=14)
plt.xlabel('Age Group')
plt.ylabel('Average CI Width')
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.show()

conn.close()

6. Yearly malnutrition change in specific countries(India, Nigeria, Brazil

In [ ]:
conn = sqlite3.connect('health_data.db')

# Query yearly malnutrition for selected countries
query = """
SELECT
    Country,
    Year,
    ROUND(AVG(Mean_Estimate), 2) AS Avg_Malnutrition
FROM malnutrition
WHERE Country IN ('India', 'Nigeria', 'Brazil')
GROUP BY Country, Year
ORDER BY Country, Year ASC;
"""

country_trends = pd.read_sql(query, conn)

print("📊 Yearly Malnutrition Trend in India, Nigeria, and Brazil:")
print(country_trends)

# Plot trend
plt.figure(figsize=(8,5))
for country in country_trends['Country'].unique():
    subset = country_trends[country_trends['Country'] == country]
    plt.plot(subset['Year'], subset['Avg_Malnutrition'], marker='o', label=country)

plt.title('Yearly Malnutrition Trend in Selected Countries', fontsize=14)
plt.xlabel('Year')
plt.ylabel('Average Malnutrition (%)')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

conn.close()

7. Regions with lowest malnutrition averages

In [ ]:
conn = sqlite3.connect('health_data.db')

# Query regions with lowest malnutrition averages
query = """
SELECT
    Region,
    ROUND(AVG(Mean_Estimate), 2) AS Avg_Malnutrition
FROM malnutrition
GROUP BY Region
ORDER BY Avg_Malnutrition ASC
LIMIT 5;
"""

lowest_regions = pd.read_sql(query, conn)

print("🌿 Regions with Lowest Average Malnutrition:")
print(lowest_regions)

# Plot bar chart
plt.figure(figsize=(8,4))
plt.bar(lowest_regions['Region'], lowest_regions['Avg_Malnutrition'], color='lightgreen')
plt.title('Regions with Lowest Average Malnutrition', fontsize=14)
plt.xlabel('Region')
plt.ylabel('Average Malnutrition (%)')
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.show()

conn.close()

8. Countries with increasing malnutrition (💡 Hint: Use MIN() and MAX()   on Mean_Estimate per country to compare early vs. recent malnutrition levels, and filter where the difference is positive using HAVING.)

In [ ]:
conn = sqlite3.connect('health_data.db')

# Query countries with increasing malnutrition
query = """
SELECT
    Country,
    ROUND(MIN(Mean_Estimate), 2) AS Min_Malnutrition,
    ROUND(MAX(Mean_Estimate), 2) AS Max_Malnutrition,
    ROUND(MAX(Mean_Estimate) - MIN(Mean_Estimate), 2) AS Increase
FROM malnutrition
GROUP BY Country
HAVING (MAX(Mean_Estimate) - MIN(Mean_Estimate)) > 0
ORDER BY Increase DESC
LIMIT 10;
"""

increasing_malnutrition = pd.read_sql(query, conn)

print("📈 Countries with Increasing Malnutrition Over the Years:")
print(increasing_malnutrition)

conn.close()

9.  Min/Max malnutrition levels year-wise comparison

In [ ]:
conn = sqlite3.connect('health_data.db')

# Query year-wise min/max malnutrition
query = """
SELECT
    Year,
    ROUND(MIN(Mean_Estimate), 2) AS Min_Malnutrition,
    ROUND(MAX(Mean_Estimate), 2) AS Max_Malnutrition
FROM malnutrition
GROUP BY Year
ORDER BY Year ASC;
"""

yearly_range = pd.read_sql(query, conn)

print("📊 Year-wise Min/Max Malnutrition Levels:")
print(yearly_range)

# Plot min/max trend
plt.figure(figsize=(8,5))
plt.plot(yearly_range['Year'], yearly_range['Min_Malnutrition'], marker='o', label='Min Malnutrition', color='green')
plt.plot(yearly_range['Year'], yearly_range['Max_Malnutrition'], marker='o', label='Max Malnutrition', color='red')
plt.fill_between(yearly_range['Year'], yearly_range['Min_Malnutrition'], yearly_range['Max_Malnutrition'], color='orange', alpha=0.2)
plt.title('Year-wise Min/Max Malnutrition Levels', fontsize=14)
plt.xlabel('Year')
plt.ylabel('Malnutrition (%)')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

conn.close()

10.  High CI_Width flags for monitoring(CI_width > 5)

In [ ]:
conn = sqlite3.connect('health_data.db')

# Query high CI_Width records
query = """
SELECT
    Country,
    Year,
    age_group,
    Indicator,
    Mean_Estimate,
    CI_Width
FROM malnutrition
WHERE CI_Width > 5
ORDER BY CI_Width DESC;
"""

high_ci_flags = pd.read_sql(query, conn)

print("⚠️ Malnutrition Records with High CI_Width (>5):")
print(high_ci_flags.head(20))  # show top 20 records for monitoring

conn.close()

🔗 Combined (5 Queries)

1. Obesity vs malnutrition comparison by country(any 5 countries)

In [ ]:
conn = sqlite3.connect('health_data.db')

# Query average obesity vs malnutrition for selected countries
query = """
SELECT
    o.Country,
    ROUND(AVG(o.Mean_Estimate), 2) AS Avg_Obesity,
    ROUND(AVG(m.Mean_Estimate), 2) AS Avg_Malnutrition
FROM obesity o
JOIN malnutrition m
    ON o.Country = m.Country
WHERE o.Country IN ('India','Nigeria','Brazil','USA','China')
GROUP BY o.Country
ORDER BY o.Country;
"""

comparison = pd.read_sql(query, conn)

print("🔗 Obesity vs Malnutrition Comparison (Selected Countries):")
print(comparison)

# Plot comparison
plt.figure(figsize=(8,5))
bar_width = 0.35
countries = comparison['Country']
index = range(len(countries))

plt.bar(index, comparison['Avg_Obesity'], bar_width, label='Obesity', color='salmon')
plt.bar([i + bar_width for i in index], comparison['Avg_Malnutrition'], bar_width, label='Malnutrition', color='skyblue')

plt.xlabel('Country')
plt.ylabel('Average Percentage (%)')
plt.title('Obesity vs Malnutrition Comparison')
plt.xticks([i + bar_width/2 for i in index], countries)
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.show()

conn.close()

2. Gender-based disparity in both obesity and malnutrition

In [ ]:
conn = sqlite3.connect('health_data.db')

# Obesity by gender
query_obesity = """
SELECT
    Gender AS Gender,
    ROUND(AVG(Mean_Estimate), 2) AS Avg_Obesity
FROM obesity
GROUP BY Gender;
"""
obesity_gender = pd.read_sql(query_obesity, conn)

# Malnutrition by gender
query_malnutrition = """
SELECT
    Indicator AS Gender,
    ROUND(AVG(Mean_Estimate), 2) AS Avg_Malnutrition
FROM malnutrition
GROUP BY Indicator;
"""
malnutrition_gender = pd.read_sql(query_malnutrition, conn)

# Merge for comparison
gender_comparison = pd.merge(obesity_gender, malnutrition_gender, on='Gender', how='outer')
print("⚖️ Gender-Based Disparity in Obesity and Malnutrition:")
print(gender_comparison)

# Plot comparison
plt.figure(figsize=(8,5))
bar_width = 0.35
genders = gender_comparison['Gender']
index = range(len(genders))

plt.bar(index, gender_comparison['Avg_Obesity'], bar_width, label='Obesity', color='salmon')
plt.bar([i + bar_width for i in index], gender_comparison['Avg_Malnutrition'], bar_width, label='Malnutrition', color='skyblue')

plt.xlabel('Gender')
plt.ylabel('Average Percentage (%)')
plt.title('Gender-Based Disparity: Obesity vs Malnutrition')
plt.xticks([i + bar_width/2 for i in index], genders)
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.show()

conn.close()

3. Region-wise avg estimates side-by-side(Africa and America)

In [ ]:
conn = sqlite3.connect('health_data.db')

# Obesity averages for Africa and Americas
query_obesity = """
SELECT
    Region,
    ROUND(AVG(Mean_Estimate), 2) AS Avg_Obesity
FROM obesity
WHERE Region IN ('Africa', 'Americas')
GROUP BY Region;
"""
obesity_region = pd.read_sql(query_obesity, conn)

# Malnutrition averages for Africa and Americas
query_malnutrition = """
SELECT
    Region,
    ROUND(AVG(Mean_Estimate), 2) AS Avg_Malnutrition
FROM malnutrition
WHERE Region IN ('Africa', 'Americas')
GROUP BY Region;
"""
malnutrition_region = pd.read_sql(query_malnutrition, conn)

# Merge for side-by-side comparison
region_comparison = pd.merge(obesity_region, malnutrition_region, on='Region')
print("🌍 Region-Wise Average Estimates: Obesity vs Malnutrition")
print(region_comparison)

# Plot comparison
plt.figure(figsize=(6,5))
bar_width = 0.35
regions = region_comparison['Region']
index = range(len(regions))

plt.bar(index, region_comparison['Avg_Obesity'], bar_width, label='Obesity', color='salmon')
plt.bar([i + bar_width for i in index], region_comparison['Avg_Malnutrition'], bar_width, label='Malnutrition', color='skyblue')

plt.xlabel('Region')
plt.ylabel('Average Percentage (%)')
plt.title('Region-Wise Average Estimates: Obesity vs Malnutrition')
plt.xticks([i + bar_width/2 for i in index], regions)
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.show()

conn.close()

4. Countries with obesity up & malnutrition down

In [ ]:
conn = sqlite3.connect('health_data.db')

query = """
WITH ObesityTrend AS (
    SELECT
        Country,
        MAX(Mean_Estimate) - MIN(Mean_Estimate) AS Obesity_Change
    FROM obesity
    GROUP BY Country
),
MalnutritionTrend AS (
    SELECT
        Country,
        MAX(Mean_Estimate) - MIN(Mean_Estimate) AS Malnutrition_Change
    FROM malnutrition
    GROUP BY Country
)
SELECT
    o.Country,
    ROUND(o.Obesity_Change, 2) AS Obesity_Change,
    ROUND(m.Malnutrition_Change, 2) AS Malnutrition_Change
FROM ObesityTrend o
JOIN MalnutritionTrend m
    ON o.Country = m.Country
WHERE o.Obesity_Change > 0 AND m.Malnutrition_Change < 0
ORDER BY o.Obesity_Change DESC
LIMIT 10;
"""

trend_comparison = pd.read_sql(query, conn)
print("📈 Countries with Increasing Obesity & Decreasing Malnutrition:")
print(trend_comparison)

conn.close()

5. Age-wise trend analysis

In [ ]:
conn = sqlite3.connect('health_data.db')

# Obesity age-wise trend
query_obesity = """
SELECT
    age_group,
    Year,
    ROUND(AVG(Mean_Estimate), 2) AS Avg_Obesity
FROM obesity
GROUP BY age_group, Year
ORDER BY age_group, Year;
"""
obesity_age_trend = pd.read_sql(query_obesity, conn)

# Malnutrition age-wise trend
query_malnutrition = """
SELECT
    age_group,
    Year,
    ROUND(AVG(Mean_Estimate), 2) AS Avg_Malnutrition
FROM malnutrition
GROUP BY age_group, Year
ORDER BY age_group, Year;
"""
malnutrition_age_trend = pd.read_sql(query_malnutrition, conn)

# Plot Obesity Trend
plt.figure(figsize=(8,5))
for age_group in obesity_age_trend['age_group'].unique():
    subset = obesity_age_trend[obesity_age_trend['age_group'] == age_group]
    plt.plot(subset['Year'], subset['Avg_Obesity'], marker='o', label=f'Obesity - {age_group}')

# Plot Malnutrition Trend
for age_group in malnutrition_age_trend['age_group'].unique():
    subset = malnutrition_age_trend[malnutrition_age_trend['age_group'] == age_group]
    plt.plot(subset['Year'], subset['Avg_Malnutrition'], marker='x', linestyle='--', label=f'Malnutrition - {age_group}')

plt.title('Age-Wise Trend Analysis: Obesity vs Malnutrition', fontsize=14)
plt.xlabel('Year')
plt.ylabel('Average Percentage (%)')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

conn.close()

### Streamlit

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import subprocess
import sys
import seaborn as sns

# --- Connect to SQLite database ---
conn = sqlite3.connect('health_data.db')

# --- Sidebar options ---
st.sidebar.title("Health Data Explorer")
dataset_choice = st.sidebar.selectbox("Select Dataset", ["Obesity", "Malnutrition"])
eda_choice = st.sidebar.selectbox("Select Analysis", [
    "Top Countries",
    "Trend Over Years",
    "Gender Comparison",
    "Age-wise Trend",
    "Region-wise Comparison",
    "Distribution Comparison", # Added distribution comparison
    "Queries"
])

# --- Function to load data ---
def load_data(table):
    return pd.read_sql(f"SELECT * FROM {table};", conn)

# --- Display dataset ---
if eda_choice != "Queries":
    st.title(f"{dataset_choice} Dataset")
    df = load_data(dataset_choice.lower())
    st.dataframe(df.head(10))


# --- Analysis: Top Countries ---
if eda_choice == "Top Countries":
    query = f"""
    SELECT Country, ROUND(AVG(Mean_Estimate), 2) AS Avg_Value
    FROM {dataset_choice.lower()}
    GROUP BY Country
    ORDER BY Avg_Value DESC
    LIMIT 10;
    """
    result = pd.read_sql(query, conn)
    st.subheader("Top 10 Countries by Average Value")
    st.dataframe(result)

# --- Analysis: Trend Over Years ---
elif eda_choice == "Trend Over Years":
    query = f"""
    SELECT Year, ROUND(AVG(Mean_Estimate), 2) AS Avg_Value
    FROM {dataset_choice.lower()}
    GROUP BY Year
    ORDER BY Year ASC;
    """
    trend = pd.read_sql(query, conn)
    st.subheader("Trend Over Years")
    st.line_chart(trend.rename(columns={"Year": "index"}).set_index("index"))

# --- Analysis: Gender Comparison ---
elif eda_choice == "Gender Comparison":
    query = f"""
    SELECT Indicator AS Gender, ROUND(AVG(Mean_Estimate), 2) AS Avg_Value
    FROM {dataset_choice.lower()}
    GROUP BY Indicator;
    """
    gender_df = pd.read_sql(query, conn)
    st.subheader("Gender-Based Comparison")
    st.bar_chart(gender_df.set_index("Gender"))

# --- Analysis: Age-wise Trend ---
elif eda_choice == "Age-wise Trend":
    query = f"""
    SELECT age_group, Year, ROUND(AVG(Mean_Estimate), 2) AS Avg_Value
    FROM {dataset_choice.lower()}
    GROUP BY age_group, Year
    ORDER BY age_group, Year;
    """
    age_trend = pd.read_sql(query, conn)
    st.subheader("Age-wise Trend Over Years")
    plt.figure(figsize=(8,5))
    for age_group in age_trend['age_group'].unique():
        subset = age_trend[age_trend['age_group'] == age_group]
        plt.plot(subset['Year'], subset['Avg_Value'], marker='o', label=age_group)
    plt.title(f"{dataset_choice} Trend by Age Group")
    plt.xlabel("Year")
    plt.ylabel("Average Percentage (%)")
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.5)
    st.pyplot(plt)

# --- Analysis: Region-wise Comparison ---
elif eda_choice == "Region-wise Comparison":
    query = f"""
    SELECT Region, ROUND(AVG(Mean_Estimate), 2) AS Avg_Value
    FROM {dataset_choice.lower()}
    GROUP BY Region
    ORDER BY Avg_Value DESC;
    """
    region_df = pd.read_sql(query, conn)
    st.subheader("Region-wise Average")
    st.bar_chart(region_df.set_index("Region"))

# --- Analysis: Distribution Comparison (Obesity vs Malnutrition) ---
elif eda_choice == "Distribution Comparison":
    st.subheader("Distribution Comparison: Obesity vs Malnutrition")
    obesity_df = load_data("obesity")
    malnutrition_df = load_data("malnutrition")

    # Add a 'Type' column to each dataframe
    obesity_df['Type'] = 'Obesity'
    malnutrition_df['Type'] = 'Malnutrition'

    # Combine the dataframes
    df_combined = pd.concat([obesity_df, malnutrition_df], ignore_index=True)

    # Histogram/KDE plot
    plt.figure(figsize=(10,6))
    sns.histplot(
        data=df_combined,
        x='Mean_Estimate',
        hue='Type',
        kde=True,
        stat='density',
        common_norm=False,
        element='step',
        alpha=0.5
    )
    plt.title('Distribution Comparison: Obesity vs Malnutrition')
    plt.xlabel('Mean Estimate (%)')
    plt.ylabel('Density')
    plt.legend(title='Type') # Added legend title
    st.pyplot(plt)

    # Boxplot
    plt.figure(figsize=(8,5))
    sns.boxplot(x='Type', y='Mean_Estimate', data=df_combined, palette=['#FF6347','#4682B4'])
    plt.title('Boxplot of Mean_Estimate by Type')
    plt.ylabel('Mean_Estimate (%)')
    st.pyplot(plt)

    # Regional Comparison
    plt.figure(figsize=(12,6))
    sns.barplot(x='Region', y='Mean_Estimate', hue='Type', data=df_combined, errorbar=None)
    plt.xticks(rotation=45)
    plt.title('Average Mean_Estimate by Region: Obesity vs Malnutrition')
    plt.ylabel('Mean_Estimate (%)')
    st.pyplot(plt)


# --- Analysis: Run Queries ---
elif eda_choice == "Queries":
    st.subheader("Run Predefined Queries")
    option = st.selectbox(
       "Select your query",[
           '1.Top 5 regions with the highest average obesity levels in the most recent year(2022)',
           '2.Top 5 countries with highest obesity estimates',
           '3.Obesity trend in India over the years(Mean_estimate)',
           '4.Average obesity by gender',
           '5.Country count by obesity level category and age group',
           '6.Top 5 countries least reliable countries(with highest CI_Width) and Top 5 most consistent countries (smallest average CI_Width)',
           '7.Average obesity by age group',
           '8.Top 10 Countries with consistent low obesity (low average + low CI)over the years',
           '9.Countries where female obesity exceeds male by large margin (same       year)',
           '10.Global average obesity percentage per year',
           '11.Avg. malnutrition by age group',
           '12.Top 5 countries with highest malnutrition(mean_estimate)',
           '13.Malnutrition trend in African region over the years',
           '14.Gender-based average malnutrition',
           '15.Malnutrition level-wise (average CI_Width by age group)',
           '16.Yearly malnutrition change in specific countries(India, Nigeria, Brazil)',
           '17.Regions with lowest malnutrition averages',
           '18.Countries with increasing malnutrition (💡 Hint: Use MIN() and MAX()   on Mean_Estimate per country to compare early vs. recent malnutrition levels, and filter where the difference is positive using HAVING.)',
           '19.Min/Max malnutrition levels year-wise comparison',
           '20. High CI_Width flags for monitoring(CI_width > 5)',
           '21.Obesity vs malnutrition comparison by country(any 5 countries)',
           '22.Gender-based disparity in both obesity and malnutrition',
           '23.Region-wise avg estimates side-by-side(Africa and America)',
           '24.Countries with obesity up & malnutrition down',
           '25.Age-wise trend analysis',
                ],
    index=None)
    if option == '1.Top 5 regions with the highest average obesity levels in the most recent year(2022)':
        conn = sqlite3.connect('health_data.db')
        query = """
        SELECT Region, AVG(Mean_Estimate) AS Avg_Obesity
        FROM obesity
        WHERE Year = 2022
        GROUP BY Region
        ORDER BY Avg_Obesity DESC
        LIMIT 5;
        """
        top5_regions = pd.read_sql(query, conn)
        st.write("🌍 **Top 5 Regions with Highest Average Obesity Levels in 2022**")
        st.dataframe(top5_regions)
        st.bar_chart(top5_regions.set_index("Region"))
        conn.close()


    if option == '2.Top 5 countries with highest obesity estimates':
        conn = sqlite3.connect('health_data.db')
        query = """
        SELECT
            Country,
            ROUND(AVG(Mean_Estimate), 2) AS Avg_Obesity_Level
        FROM obesity
        GROUP BY Country
        ORDER BY Avg_Obesity_Level DESC
        LIMIT 5;
        """
        top5_countries = pd.read_sql(query, conn)
        st.write("🌍 Top 5 Countries with Highest Average Obesity Estimates:")
        st.dataframe(top5_countries)
        conn.close()

    if option == '3.Obesity trend in India over the years(Mean_estimate)':
        conn = sqlite3.connect('health_data.db')
        query = """
        SELECT
            Year,
            ROUND(AVG(Mean_Estimate), 2) AS Avg_Obesity_Level
        FROM obesity
        WHERE Country = 'India'
        GROUP BY Year
        ORDER BY Year ASC;
        """
        india_trend = pd.read_sql(query, conn)
        st.write("📊 Obesity Trend in India Over the Years:")
        st.dataframe(india_trend)
        conn.close()


    if option == '4.Average obesity by gender':
        conn = sqlite3.connect('health_data.db')
        query = """
        SELECT
            Gender,
            ROUND(AVG(Mean_Estimate), 2) AS Avg_Obesity_Level
        FROM obesity
        GROUP BY Gender
        ORDER BY Avg_Obesity_Level DESC;
        """
        gender_obesity = pd.read_sql(query, conn)
        st.write("⚖️ Average Obesity by Gender:")
        st.dataframe(gender_obesity)
        plt.figure(figsize=(6,4))
        plt.bar(gender_obesity['Gender'], gender_obesity['Avg_Obesity_Level'], color='skyblue')
        plt.title('Average Obesity by Gender', fontsize=14)
        plt.xlabel('Gender')
        plt.ylabel('Average Obesity Level (%)')
        plt.grid(axis='y', linestyle='--', alpha=0.6)
        st.pyplot(plt)
        conn.close()


    if option == '5.Country count by obesity level category and age group':
        conn = sqlite3.connect('health_data.db')

        query = """
        WITH categorized AS (
            SELECT
                Country,
                age_group,
                CASE
                    WHEN Mean_Estimate < 10 THEN 'Low'
                    WHEN Mean_Estimate BETWEEN 10 AND 20 THEN 'Moderate'
                    WHEN Mean_Estimate BETWEEN 20 AND 30 THEN 'High'
                    ELSE 'Very High'
                END AS Obesity_Level_Category
            FROM obesity
            WHERE Year = 2022
        )
        SELECT
            Obesity_Level_Category,
            age_group,
            COUNT(DISTINCT Country) AS Country_Count
        FROM categorized
        GROUP BY Obesity_Level_Category, age_group
        ORDER BY
            CASE
                WHEN Obesity_Level_Category = 'Low' THEN 1
                WHEN Obesity_Level_Category = 'Moderate' THEN 2
                WHEN Obesity_Level_Category = 'High' THEN 3
                WHEN Obesity_Level_Category = 'Very High' THEN 4
            END,
            age_group;
        """

        obesity_category = pd.read_sql(query, conn)

        st.write("🏳️ Country Count by Obesity Level Category and Age Group (2022):")
        st.dataframe(obesity_category)
        plt.figure(figsize=(8,5))
        for group in obesity_category['age_group'].unique():
            subset = obesity_category[obesity_category['age_group'] == group]
            plt.bar(subset['Obesity_Level_Category'], subset['Country_Count'], label=group)

        plt.title('Country Count by Obesity Level Category and Age Group (2022)')
        plt.xlabel('Obesity Level Category')
        plt.ylabel('Number of Countries')
        plt.legend(title='Age Group')
        plt.grid(axis='y', linestyle='--', alpha=0.6)
        st.pyplot(plt)

        conn.close()


    if option == '6.Top 5 countries least reliable countries(with highest CI_Width) and Top 5 most consistent countries (smallest average CI_Width)':
        conn = sqlite3.connect('health_data.db')

        query_least_reliable = """
        SELECT
            Country,
            ROUND(AVG(CI_Width), 3) AS Avg_CI_Width
        FROM obesity
        GROUP BY Country
        ORDER BY Avg_CI_Width DESC
        LIMIT 5;
        """

        least_reliable = pd.read_sql(query_least_reliable, conn)

        # Top 5 most consistent (lowest CI_Width)
        query_most_consistent = """
        SELECT
            Country,
            ROUND(AVG(CI_Width), 3) AS Avg_CI_Width
        FROM obesity
        GROUP BY Country
        ORDER BY Avg_CI_Width ASC
        LIMIT 5;
        """

        most_consistent = pd.read_sql(query_most_consistent, conn)

        st.write("⚠️ Top 5 Least Reliable Countries (Highest Average CI_Width):")
        st.dataframe(least_reliable)
        st.write("✅ Top 5 Most Consistent Countries (Lowest Average CI_Width):")
        st.dataframe(most_consistent)

        conn.close()


    if option == '7.Average obesity by age group':
        conn = sqlite3.connect('health_data.db')
        query = """
        SELECT
            age_group,
            ROUND(AVG(Mean_Estimate), 2) AS Avg_Obesity_Level
        FROM obesity
        GROUP BY age_group
        ORDER BY Avg_Obesity_Level DESC;
        """

        age_obesity = pd.read_sql(query, conn)

        plt.figure(figsize=(6, 4))
        plt.bar(age_obesity['age_group'], age_obesity['Avg_Obesity_Level'], color=['lightcoral', 'skyblue'])
        plt.title('Average Obesity by Age Group', fontsize=14)
        plt.xlabel('Age Group')
        plt.ylabel('Average Obesity Level (%)')
        plt.grid(axis='y', linestyle='--', alpha=0.6)
        st.pyplot(plt)

        conn.close()
    if option == '8.Top 10 Countries with consistent low obesity (low average + low CI)over the years':
        conn = sqlite3.connect('health_data.db')

        # Query top 10 countries with consistently low obesity & low CI_Width
        query = """
        SELECT
            Country,
            ROUND(AVG(Mean_Estimate), 2) AS Avg_Obesity_Level,
            ROUND(AVG(CI_Width), 3) AS Avg_CI_Width
        FROM obesity
        GROUP BY Country
        HAVING Avg_Obesity_Level IS NOT NULL
        ORDER BY Avg_Obesity_Level ASC, Avg_CI_Width ASC
        LIMIT 10;
        """

        consistent_low_obesity = pd.read_sql(query, conn)

        st.write("🌿 Top 10 Countries with Consistent Low Obesity and High Data Reliability:")
        st.dataframe(consistent_low_obesity)

        # Visualization
        plt.figure(figsize=(8,5))
        plt.scatter(
            consistent_low_obesity['Avg_Obesity_Level'],
            consistent_low_obesity['Avg_CI_Width'],
            color='mediumseagreen',
            s=80
        )
        for i, row in consistent_low_obesity.iterrows():
            plt.text(row['Avg_Obesity_Level']+0.1, row['Avg_CI_Width']+0.001, row['Country'], fontsize=9)

        plt.title('Top 10 Consistently Low-Obesity Countries (Low Mean + Low CI)', fontsize=13)
        plt.xlabel('Average Obesity Level (%)')
        plt.ylabel('Average CI Width')
        plt.grid(True, linestyle='--', alpha=0.6)
        st.pyplot(plt)

        conn.close()

    if option == '9.Countries where female obesity exceeds male by large margin (same       year)':
        conn = sqlite3.connect('health_data.db')

        query = """
        SELECT
            f.Country,
            f.Year,
            ROUND(f.Mean_Estimate, 2) AS Female_Obesity,
            ROUND(m.Mean_Estimate, 2) AS Male_Obesity,
            ROUND(f.Mean_Estimate - m.Mean_Estimate, 2) AS Difference
        FROM obesity f
        JOIN obesity m
            ON f.Country = m.Country
            AND f.Year = m.Year
        WHERE f.Gender LIKE '%FMLE%'
          AND m.Gender LIKE '%MLE%'
          AND (f.Mean_Estimate - m.Mean_Estimate) > 5
        ORDER BY Difference DESC
        LIMIT 10;
        """

        gender_gap = pd.read_sql(query, conn)

        st.write("💃 Countries Where Female Obesity Exceeds Male by Large Margin:")
        st.dataframe(gender_gap)

        conn.close()

    if option == '10.Global average obesity percentage per year':
        conn = sqlite3.connect('health_data.db')

        # Query global average obesity per year
        query = """
        SELECT
            Year,
            ROUND(AVG(Mean_Estimate), 2) AS Global_Avg_Obesity
        FROM obesity
        GROUP BY Year
        ORDER BY Year ASC;
        """

        global_trend = pd.read_sql(query, conn)

        st.write("🌍 Global Average Obesity Percentage Per Year:")
        st.dataframe(global_trend)

        # Plot trend
        plt.figure(figsize=(8,5))
        plt.plot(global_trend['Year'], global_trend['Global_Avg_Obesity'], marker='o', color='mediumblue')
        plt.title('Global Average Obesity Percentage Per Year', fontsize=14)
        plt.xlabel('Year')
        plt.ylabel('Average Obesity (%)')
        plt.grid(True)
        st.pyplot(plt)

        conn.close()


    if option == '11.Avg. malnutrition by age group':
        conn = sqlite3.connect('health_data.db')

        # Query average malnutrition by age group
        query = """
        SELECT
            age_group,
            ROUND(AVG(Mean_Estimate), 2) AS Avg_Malnutrition
        FROM malnutrition
        GROUP BY age_group
        ORDER BY Avg_Malnutrition DESC;
        """

        age_malnutrition = pd.read_sql(query, conn)

        st.write("👶🧍 Average Malnutrition by Age Group:")
        st.dataframe(age_malnutrition)

        # Plot visualization
        plt.figure(figsize=(6,4))
        plt.bar(age_malnutrition['age_group'], age_malnutrition['Avg_Malnutrition'], color=['orange','skyblue'])
        plt.title('Average Malnutrition by Age Group', fontsize=14)
        plt.xlabel('Age Group')
        plt.ylabel('Average Malnutrition (%)')
        plt.grid(axis='y', linestyle='--', alpha=0.6)
        st.pyplot(plt)

        conn.close()
    if option == '12.Top 5 countries with highest malnutrition(mean_estimate)':
        conn = sqlite3.connect('health_data.db')

        # Query top 5 countries with highest malnutrition
        query = """
        SELECT
            Country,
            ROUND(AVG(Mean_Estimate), 2) AS Avg_Malnutrition
        FROM malnutrition
        GROUP BY Country
        ORDER BY Avg_Malnutrition DESC
        LIMIT 5;
        """

        top5_malnutrition = pd.read_sql(query, conn)

        st.write("⚠️ Top 5 Countries with Highest Malnutrition Rates:")
        st.dataframe(top5_malnutrition)

        conn.close()
    if option == '13.Malnutrition trend in African region over the years':
        conn = sqlite3.connect('health_data.db')

        # Query African malnutrition trend
        query = """
        SELECT
            Year,
            ROUND(AVG(Mean_Estimate), 2) AS Avg_Malnutrition
        FROM malnutrition
        WHERE Region LIKE '%Africa%'
        GROUP BY Year
        ORDER BY Year ASC;
        """

        africa_trend = pd.read_sql(query, conn)

        st.write("📊 Malnutrition Trend in African Region Over the Years:")
        st.dataframe(africa_trend)

        # Plot trend
        plt.figure(figsize=(8,5))
        plt.plot(africa_trend['Year'], africa_trend['Avg_Malnutrition'], marker='o', color='orangered')
        plt.title('Malnutrition Trend in African Region Over the Years', fontsize=14)
        plt.xlabel('Year')
        plt.ylabel('Average Malnutrition (%)')
        plt.grid(True)
        st.pyplot(plt)

        conn.close()
    if option == '14.Gender-based average malnutrition':
        conn = sqlite3.connect('health_data.db')

        # Query average malnutrition by gender
        query = """
        SELECT
            Indicator AS Gender,
            ROUND(AVG(Mean_Estimate), 2) AS Avg_Malnutrition
        FROM malnutrition
        GROUP BY Indicator
        ORDER BY Avg_Malnutrition DESC;
        """

        gender_malnutrition = pd.read_sql(query, conn)

        st.write("⚖️ Average Malnutrition by Gender:")
        st.dataframe(gender_malnutrition)

        # Plot bar chart
        plt.figure(figsize=(6,4))
        plt.bar(gender_malnutrition['Gender'], gender_malnutrition['Avg_Malnutrition'], color=['orange','skyblue'])
        plt.title('Average Malnutrition by Gender', fontsize=14)
        plt.xlabel('Gender')
        plt.ylabel('Average Malnutrition (%)')
        plt.grid(axis='y', linestyle='--', alpha=0.6)
        st.pyplot(plt)

        conn.close()
    if option == '15.Malnutrition level-wise (average CI_Width by age group)':
        conn = sqlite3.connect('health_data.db')

        # Query average CI_Width by age group
        query = """
        SELECT
            age_group,
            ROUND(AVG(CI_Width), 3) AS Avg_CI_Width
        FROM malnutrition
        GROUP BY age_group
        ORDER BY Avg_CI_Width ASC;
        """

        ci_by_age = pd.read_sql(query, conn)

        st.write("📏 Average CI_Width by Age Group (Malnutrition):")
        st.dataframe(ci_by_age)

        # Plot
        plt.figure(figsize=(6,4))
        plt.bar(ci_by_age['age_group'], ci_by_age['Avg_CI_Width'], color=['lightgreen','salmon'])
        plt.title('Average CI_Width by Age Group (Malnutrition)', fontsize=14)
        plt.xlabel('Age Group')
        plt.ylabel('Average CI Width')
        plt.grid(axis='y', linestyle='--', alpha=0.6)
        st.pyplot(plt)

        conn.close()
    if option == '16.Yearly malnutrition change in specific countries(India, Nigeria, Brazil)':
        conn = sqlite3.connect('health_data.db')

        # Query yearly malnutrition for selected countries
        query = """
        SELECT
            Country,
            Year,
            ROUND(AVG(Mean_Estimate), 2) AS Avg_Malnutrition
        FROM malnutrition
        WHERE Country IN ('India', 'Nigeria', 'Brazil')
        GROUP BY Country, Year
        ORDER BY Country, Year ASC;
        """

        country_trends = pd.read_sql(query, conn)

        st.write("📊 Yearly Malnutrition Trend in India, Nigeria, and Brazil:")
        st.dataframe(country_trends)

        # Plot trend
        plt.figure(figsize=(8,5))
        for country in country_trends['Country'].unique():
            subset = country_trends[country_trends['Country'] == country]
            plt.plot(subset['Year'], subset['Avg_Malnutrition'], marker='o', label=country)

        plt.title('Yearly Malnutrition Trend in Selected Countries', fontsize=14)
        plt.xlabel('Year')
        plt.ylabel('Average Malnutrition (%)')
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.6)
        st.pyplot(plt)

        conn.close()
    if option == '17.Regions with lowest malnutrition averages':
        conn = sqlite3.connect('health_data.db')

        # Query regions with lowest malnutrition averages
        query = """
        SELECT
            Region,
            ROUND(AVG(Mean_Estimate), 2) AS Avg_Malnutrition
        FROM malnutrition
        GROUP BY Region
        ORDER BY Avg_Malnutrition ASC
        LIMIT 5;
        """

        lowest_regions = pd.read_sql(query, conn)

        st.write("🌿 Regions with Lowest Average Malnutrition:")
        st.dataframe(lowest_regions)

        # Plot bar chart
        plt.figure(figsize=(8,4))
        plt.bar(lowest_regions['Region'], lowest_regions['Avg_Malnutrition'], color='lightgreen')
        plt.title('Regions with Lowest Average Malnutrition', fontsize=14)
        plt.xlabel('Region')
        plt.ylabel('Average Malnutrition (%)')
        plt.grid(axis='y', linestyle='--', alpha=0.6)
        st.pyplot(plt)

        conn.close()
    if option == '18.Countries with increasing malnutrition (💡 Hint: Use MIN() and MAX()   on Mean_Estimate per country to compare early vs. recent malnutrition levels, and filter where the difference is positive using HAVING.)':
        conn = sqlite3.connect('health_data.db')

        query = """
        SELECT
            Country,
            ROUND(MIN(Mean_Estimate), 2) AS Min_Malnutrition,
            ROUND(MAX(Mean_Estimate), 2) AS Max_Malnutrition,
            ROUND(MAX(Mean_Estimate) - MIN(Mean_Estimate), 2) AS Increase
        FROM malnutrition
        GROUP BY Country
        HAVING (MAX(Mean_Estimate) - MIN(Mean_Estimate)) > 0
        ORDER BY Increase DESC
        LIMIT 10;
        """

        increasing_malnutrition = pd.read_sql(query, conn)

        st.write("📈 Countries with Increasing Malnutrition Over the Years:")
        st.dataframe(increasing_malnutrition)

        conn.close()
    if option == '19.Min/Max malnutrition levels year-wise comparison':
        conn = sqlite3.connect('health_data.db')

        # Query year-wise min/max malnutrition
        query = """
        SELECT
            Year,
            ROUND(MIN(Mean_Estimate), 2) AS Min_Malnutrition,
            ROUND(MAX(Mean_Estimate), 2) AS Max_Malnutrition
        FROM malnutrition
        GROUP BY Year
        ORDER BY Year ASC;
        """

        yearly_range = pd.read_sql(query, conn)

        st.write("📊 Year-wise Min/Max Malnutrition Levels:")
        st.dataframe(yearly_range)

        # Plot min/max trend
        plt.figure(figsize=(8,5))
        plt.plot(yearly_range['Year'], yearly_range['Min_Malnutrition'], marker='o', label='Min Malnutrition', color='green')
        plt.plot(yearly_range['Year'], yearly_range['Max_Malnutrition'], marker='o', label='Max Malnutrition', color='red')
        plt.fill_between(yearly_range['Year'], yearly_range['Min_Malnutrition'], yearly_range['Max_Malnutrition'], color='orange', alpha=0.2)
        plt.title('Year-wise Min/Max Malnutrition Levels', fontsize=14)
        plt.xlabel('Year')
        plt.ylabel('Malnutrition (%)')
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.6)
        st.pyplot(plt)

        conn.close()
    if option == '20. High CI_Width flags for monitoring(CI_width > 5)':
        conn = sqlite3.connect('health_data.db')

        # Query high CI_Width records
        query = """
        SELECT
            Country,
            Year,
            age_group,
            Indicator,
            Mean_Estimate,
            CI_Width
        FROM malnutrition
        WHERE CI_Width > 5
        ORDER BY CI_Width DESC;
        """

        high_ci_flags = pd.read_sql(query, conn)

        st.write("⚠️ Malnutrition Records with High CI_Width (>5):")
        st.dataframe(high_ci_flags.head(20))  # show top 20 records for monitoring

        conn.close()
    if option == '21.Obesity vs malnutrition comparison by country(any 5 countries)':
        conn = sqlite3.connect('health_data.db')

        # Query average obesity vs malnutrition for selected countries
        query = """
        SELECT
            o.Country,
            ROUND(AVG(o.Mean_Estimate), 2) AS Avg_Obesity,
            ROUND(AVG(m.Mean_Estimate), 2) AS Avg_Malnutrition
        FROM obesity o
        JOIN malnutrition m
            ON o.Country = m.Country
        WHERE o.Country IN ('India','Nigeria','Brazil','USA','China')
        GROUP BY o.Country
        ORDER BY o.Country;
        """

        comparison = pd.read_sql(query, conn)

        st.write("🔗 Obesity vs Malnutrition Comparison (Selected Countries):")
        st.dataframe(comparison)

        # Plot comparison
        plt.figure(figsize=(8,5))
        bar_width = 0.35
        countries = comparison['Country']
        index = range(len(countries))

        plt.bar(index, comparison['Avg_Obesity'], bar_width, label='Obesity', color='salmon')
        plt.bar([i + bar_width for i in index], comparison['Avg_Malnutrition'], bar_width, label='Malnutrition', color='skyblue')

        plt.xlabel('Country')
        plt.ylabel('Average Percentage (%)')
        plt.title('Obesity vs Malnutrition Comparison')
        plt.xticks([i + bar_width/2 for i in index], countries)
        plt.legend()
        plt.grid(axis='y', linestyle='--', alpha=0.6)
        st.pyplot(plt)

        conn.close()
    if option == '22.Gender-based disparity in both obesity and malnutrition':
        conn = sqlite3.connect('health_data.db')

        # Obesity by gender
        query_obesity = """
        SELECT
            Gender AS Gender,
            ROUND(AVG(Mean_Estimate), 2) AS Avg_Obesity
        FROM obesity
        GROUP BY Gender;
        """
        obesity_gender = pd.read_sql(query_obesity, conn)

        # Malnutrition by gender
        query_malnutrition = """
        SELECT
            Indicator AS Gender,
            ROUND(AVG(Mean_Estimate), 2) AS Avg_Malnutrition
        FROM malnutrition
        GROUP BY Indicator;
        """
        malnutrition_gender = pd.read_sql(query_malnutrition, conn)

        # Merge for comparison
        gender_comparison = pd.merge(obesity_gender, malnutrition_gender, on='Gender', how='outer')
        st.write("⚖️ Gender-Based Disparity in Obesity and Malnutrition:")
        st.dataframe(gender_comparison)

        # Plot comparison
        plt.figure(figsize=(8,5))
        bar_width = 0.35
        genders = gender_comparison['Gender']
        index = range(len(genders))

        plt.bar(index, gender_comparison['Avg_Obesity'], bar_width, label='Obesity', color='salmon')
        plt.bar([i + bar_width for i in index], gender_comparison['Avg_Malnutrition'], bar_width, label='Malnutrition', color='skyblue')

        plt.xlabel('Gender')
        plt.ylabel('Average Percentage (%)')
        plt.title('Gender-Based Disparity: Obesity vs Malnutrition')
        plt.xticks([i + bar_width/2 for i in index], genders)
        plt.legend()
        plt.grid(axis='y', linestyle='--', alpha=0.6)
        st.pyplot(plt)

        conn.close()
    if option == '23.Region-wise avg estimates side-by-side(Africa and America)':
        conn = sqlite3.connect('health_data.db')

        # Obesity averages for Africa and Americas
        query_obesity = """
        SELECT
            Region,
            ROUND(AVG(Mean_Estimate), 2) AS Avg_Obesity
        FROM obesity
        WHERE Region IN ('Africa', 'Americas')
        GROUP BY Region;
        """
        obesity_region = pd.read_sql(query_obesity, conn)

        # Malnutrition averages for Africa and Americas
        query_malnutrition = """
        SELECT
            Region,
            ROUND(AVG(Mean_Estimate), 2) AS Avg_Malnutrition
        FROM malnutrition
        WHERE Region IN ('Africa', 'Americas')
        GROUP BY Region;
        """
        malnutrition_region = pd.read_sql(query_malnutrition, conn)

        # Merge for side-by-side comparison
        region_comparison = pd.merge(obesity_region, malnutrition_region, on='Region')
        st.write("🌍 Region-Wise Average Estimates: Obesity vs Malnutrition")
        st.dataframe(region_comparison)

        # Plot comparison
        plt.figure(figsize=(6,5))
        bar_width = 0.35
        regions = region_comparison['Region']
        index = range(len(regions))

        plt.bar(index, region_comparison['Avg_Obesity'], bar_width, label='Obesity', color='salmon')
        plt.bar([i + bar_width for i in index], region_comparison['Avg_Malnutrition'], bar_width, label='Malnutrition', color='skyblue')

        plt.xlabel('Region')
        plt.ylabel('Average Percentage (%)')
        plt.title('Region-Wise Average Estimates: Obesity vs Malnutrition')
        plt.xticks([i + bar_width/2 for i in index], regions)
        plt.legend()
        plt.grid(axis='y', linestyle='--', alpha=0.6)
        st.pyplot(plt)

        conn.close()
    if option == '24.Countries with obesity up & malnutrition down':
        conn = sqlite3.connect('health_data.db')

        query = """
        WITH ObesityTrend AS (
            SELECT
                Country,
                MAX(Mean_Estimate) - MIN(Mean_Estimate) AS Obesity_Change
            FROM obesity
            GROUP BY Country
        ),
        MalnutritionTrend AS (
            SELECT
                Country,
                MAX(Mean_Estimate) - MIN(Mean_Estimate) AS Malnutrition_Change
            FROM malnutrition
            GROUP BY Country
        )
        SELECT
            o.Country,
            ROUND(o.Obesity_Change, 2) AS Obesity_Change,
            ROUND(m.Malnutrition_Change, 2) AS Malnutrition_Change
        FROM ObesityTrend o
        JOIN MalnutritionTrend m
            ON o.Country = m.Country
        WHERE o.Obesity_Change > 0 AND m.Malnutrition_Change < 0
        ORDER BY o.Obesity_Change DESC
        LIMIT 10;
        """

        trend_comparison = pd.read_sql(query, conn)
        st.write("📈 Countries with Increasing Obesity & Decreasing Malnutrition:")
        st.dataframe(trend_comparison)

        conn.close()
    if option == '25.Age-wise trend analysis':
        conn = sqlite3.connect('health_data.db')

        # Obesity age-wise trend
        query_obesity = """
        SELECT
            age_group,
            Year,
            ROUND(AVG(Mean_Estimate), 2) AS Avg_Obesity
        FROM obesity
        GROUP BY age_group, Year
        ORDER BY age_group, Year;
        """
        obesity_age_trend = pd.read_sql(query_obesity, conn)

        # Malnutrition age-wise trend
        query_malnutrition = """
        SELECT
            age_group,
            Year,
            ROUND(AVG(Mean_Estimate), 2) AS Avg_Malnutrition
        FROM malnutrition
        GROUP BY age_group, Year
        ORDER BY age_group, Year;
        """
        malnutrition_age_trend = pd.read_sql(query_malnutrition, conn)

        # Plot Obesity Trend
        plt.figure(figsize=(8,5))
        for age_group in obesity_age_trend['age_group'].unique():
            subset = obesity_age_trend[obesity_age_trend['age_group'] == age_group]
            plt.plot(subset['Year'], subset['Avg_Obesity'], marker='o', label=f'Obesity - {age_group}')

        # Plot Malnutrition Trend
        for age_group in malnutrition_age_trend['age_group'].unique():
            subset = malnutrition_age_trend[malnutrition_age_trend['age_group'] == age_group]
            plt.plot(subset['Year'], subset['Avg_Malnutrition'], marker='x', linestyle='--', label=f'Malnutrition - {age_group}')

        plt.title('Age-Wise Trend Analysis: Obesity vs Malnutrition', fontsize=14)
        plt.xlabel('Year')
        plt.ylabel('Average Percentage (%)')
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.6)
        st.pyplot(plt)

        conn.close()
    elif option == 'explain one more time':
        pass

In [ ]:
# @title Setup code
!pip install -q streamlit
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
import subprocess
subprocess.Popen(["./cloudflared-linux-amd64", "tunnel", "--url", "http://localhost:8501"])
!nohup /content/cloudflared-linux-amd64 tunnel --url http://localhost:8501 &

In [ ]:
!streamlit run /content/app.py &>/content/logs.txt &

In [ ]:
!grep -o 'https://.*\.trycloudflare.com' nohup.out | head -n 1 | xargs -I {} echo "Your tunnel url {}"